# Ross Ice Shelf Figures

# Import packages

In [ ]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2
%load_ext snakeviz

import itertools
import string

import geopandas as gpd
import numpy as np
import pandas as pd
import pygmt
import seaborn as sns
import verde as vd
import xarray as xr
import zarr
from polartoolkit import fetch, maps, profiles, regions, utils

import RIS_gravity_inversion.gravity_processing as grav
import RIS_gravity_inversion.uncertainties as uncert
import RIS_gravity_inversion.utils as inv_utils

sns.set_theme()

# make beep sound when notify is called at end of long-running cells
from IPython.display import Audio

wave = np.sin(2 * np.pi * 400 * np.arange(10000 * 1) / 10000)
notify = Audio(wave, rate=10000, autoplay=True)

In [ ]:
# from pygmt import session_management

# session_management.end()
# session_management.begin()

# Set region and parameters

In [ ]:
starting_region = [-580e3, 420e3, -1420e3, -420e3]

inversion_region = starting_region
buffer_region = vd.pad_region(starting_region, 40e3)

# set layer parameters
layer_spacing = 5e3
registration = "g"

# set gravity observation parameters
grav_observation_height = 1e3
grav_spacing = 5e3

print(f"layer spacing: {layer_spacing}m")
print(f"registration: {registration}")
print("inversion region = ", inversion_region)
print("buffer region = ", buffer_region)

# Load data

## bed constraints
from notebook `RIS_bathymetry_data.ipynb`

In [ ]:
constraints_fname = "../../data/bed_constraints.csv.gz"

constraints = pd.read_csv(
    constraints_fname,
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

# ensure all points are within buffer region
constraints = utils.points_inside_region(
    constraints,
    # buffer_region,
    inversion_region,
    names=("easting", "northing"),
)

print(f"bathymetry control points:{len(constraints)}")

In [ ]:
print("Number of constraints inside RIS:", len(constraints[constraints.inside]))

## topographic layers
* Surface, Icebase, Bed

From notebook `prepare_grids`

### Get Bedmachine v3 data

In [ ]:
# surface
fname = "../../inversion_layers/bedmachine_surface.zarr"
bedmachine_surface_full = xr.open_zarr(fname).surface
# icebase
fname = "../../inversion_layers/bedmachine_icebase.zarr"
bedmachine_icebase_full = xr.open_zarr(fname).icebase
# bed
fname = "../../inversion_layers/bedmachine_bed.zarr"
bedmachine_bed_full = xr.open_zarr(fname).bed
bedmachine_bed_antarctica = xr.open_zarr(
    "../../inversion_layers/bedmachine_bed_full.zarr"
)["bed"]
# error
bedmachine_bed_error_full = xr.open_zarr(
    "../../inversion_layers/bedmachine_bed_error_full.zarr"
)["bed_error"]

# resample bedmachine data at new spacing
bedmachine_surface_1k = fetch.resample_grid(
    bedmachine_surface_full,
    spacing=1e3,
    registration=registration,
    region=buffer_region,
    verbose="q",
)
bedmachine_icebase_1k = fetch.resample_grid(
    bedmachine_icebase_full,
    spacing=1e3,
    registration=registration,
    region=buffer_region,
    verbose="q",
)
bedmachine_bed_1k = fetch.resample_grid(
    bedmachine_bed_full,
    spacing=1e3,
    registration=registration,
    region=buffer_region,
    verbose="q",
)

# resample bedmachine data at layer spacing
bedmachine_surface_layer_spacing = fetch.resample_grid(
    bedmachine_surface_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bedmachine_icebase_layer_spacing = fetch.resample_grid(
    bedmachine_icebase_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bedmachine_bed_layer_spacing = fetch.resample_grid(
    bedmachine_bed_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
# ensure layers never cross
bedmachine_icebase_layer_spacing = xr.where(
    bedmachine_icebase_layer_spacing > bedmachine_surface_layer_spacing,
    bedmachine_surface_layer_spacing,
    bedmachine_icebase_layer_spacing,
)

### Get Bedmap2 data

In [ ]:
# surface
fname = "../../inversion_layers/bedmap2_surface.zarr"
bedmap2_surface_full = xr.open_zarr(fname).surface
# icebase
fname = "../../inversion_layers/bedmap2_icebase.zarr"
bedmap2_icebase_full = xr.open_zarr(fname).icebase
# bed
fname = "../../inversion_layers/bedmap2_bed.zarr"
bedmap2_bed_full = xr.open_zarr(fname).bed

# resample bedmap2 data at new spacing
bedmap2_surface_1k = fetch.resample_grid(
    bedmap2_surface_full,
    spacing=1e3,
    registration=registration,
    region=buffer_region,
    verbose="q",
)
bedmap2_icebase_1k = fetch.resample_grid(
    bedmap2_icebase_full,
    spacing=1e3,
    registration=registration,
    region=buffer_region,
    verbose="q",
)
bedmap2_bed_1k = fetch.resample_grid(
    bedmap2_bed_full,
    spacing=1e3,
    registration=registration,
    region=buffer_region,
    verbose="q",
)

# resample bedmachine data at layer spacing
bedmap2_surface_layer_spacing = fetch.resample_grid(
    bedmap2_surface_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bedmap2_icebase_layer_spacing = fetch.resample_grid(
    bedmap2_icebase_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
bedmap2_bed_layer_spacing = fetch.resample_grid(
    bedmap2_bed_full,
    spacing=layer_spacing,
    region=buffer_region,
    registration=registration,
    verbose="q",
)
# ensure layers never cross
bedmap2_icebase_layer_spacing = xr.where(
    bedmap2_icebase_layer_spacing > bedmap2_surface_layer_spacing,
    bedmap2_surface_layer_spacing,
    bedmap2_icebase_layer_spacing,
)

### Starting bed models

In [ ]:
# starting bed
bed_from_constraints_mean = xr.open_zarr(
    "../../inversion_layers/RIS_starting_bed_stats.zarr",
).weighted_mean
bed_from_constraints_verde_spline = xr.open_zarr(
    "../../inversion_layers/bed_from_constraints_verde_spline.zarr",
).bed
bed_from_constraints_gmt_surface = xr.open_zarr(
    "../../inversion_layers/bed_from_constraints_gmt_surface.zarr",
).bed

# BASEMENT
# Ross Embayment basement merged to bedmachine bed
# basement = "inversion_layers/bed_basement_merged.zarr"
# Ross Embayment basement merged to ANTASed basement
# basement = "inversion_layers/ANTASed_magnetics_merged_basement.zarr"
# basement = xr.open_zarr(basement).basement

starting_bed_prisms = xr.open_zarr(
    "../../inversion_layers/RIS_starting_bed_prisms_mean.zarr",
    # "../../inversion_layers/RIS_starting_bed_prisms_verde_spline.zarr",
    # "../../inversion_layers/RIS_starting_bed_prisms_gmt_surface.zarr",
)

min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
    mindist=layer_spacing / np.sqrt(2),
    low=0,
    high=1,
    region=inversion_region,
)
starting_bed_prisms["weights"] = min_dist

## gravity data

Get levelled points, and gridded dataframe with disturbance, and terrain mass effects

In [ ]:
rosetta_levelled = pd.read_csv(
    "../../data/ROSETTA_2nd_order_levelled.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
# rename second to last column to "disturbance"
col = list(rosetta_levelled.columns)[-2]
rosetta_levelled = rosetta_levelled.rename(columns={col: "disturbance"})


df_anomalies = pd.read_csv(
    "../../data/RIS_topo_corrections_levelled_mean_bed.csv.gz",
    # "../../data/RIS_topo_corrections_levelled_verde_spline.csv.gz",
    # "../../data/RIS_topo_corrections_levelled_gmt_surface.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)

In [ ]:
print(len(df_anomalies))
print(len(df_anomalies[df_anomalies.test]))
print(len(df_anomalies[~df_anomalies.test]))

## Starting bed models

In [ ]:
combined = xr.open_zarr("../../inversion_layers/RIS_starting_bed_models.zarr")
combined_list = [combined[i] for i in combined]

In [ ]:
stats = xr.open_zarr("../../inversion_layers/RIS_starting_bed_stats.zarr")

## Inversion results

In [ ]:
prism_results = xr.open_zarr("../../results/RIS_inverted_bed.zarr")
final_bed = prism_results.final_bed

In [ ]:
grav_results = xr.open_zarr("../../results/RIS_inverted_bed_grav_results.zarr")

## Monte Carlo results

In [ ]:
path = "../../results/monte_carlo_results/"

prefixes = [
    "RIS_monte_carlo_gravity",
    "RIS_monte_carlo_constraints",
    "RIS_monte_carlo_densities",
    "RIS_monte_carlo_starting_bed_tension",
    "RIS_monte_carlo_regional_tension",
    "RIS_monte_carlo_full_gmt_surface",
    "RIS_monte_carlo_full_gmt_surface",
]

MCstats = []
for i, j in enumerate(prefixes):
    result = uncert.monte_carlo_full_workflow_uncertainty_loop(
        fname=path + j,
        runs=10,
        grav=df_anomalies,
        constraints=constraints,
    )
    ds = uncert.merged_stats(
        result,
        "topo",
        -1,
        plot=False,
        constraints=constraints,
        weight_by="constraints",
    )

    if i == 6:
        MCstats.append(ds.weighted_mean)
    else:
        MCstats.append(ds.weighted_stdev)

### Low pass filter

In [ ]:
filtered_15k_MC_stats = []
for i, j in enumerate(MCstats):
    if i == 6:
        # low pass filter bed to remove some of the noise
        filtered = inv_utils.filter_grid(
            j,
            filter_width=15e3,
            filt_type="lowpass",
        )
        # mask bed outside of RIS
        filtered = utils.mask_from_shp(
            "../../plotting/RIS_outline.shp",
            xr_grid=filtered,
            masked=True,
            invert=False,
        )
        # merge back with unfiltered
        filtered = xr.where(
            filtered.isnull(),
            j,
            filtered,
        )
    else:
        filtered = inv_utils.filter_grid(
            j,
            filter_width=15e3,
            filt_type="lowpass",
        )

    filtered_15k_MC_stats.append(filtered)

    _ = utils.grd_compare(
        j,
        filtered,
        grid1_name="Unfiltered",
        grid2_name="Filtered",
        title="difference",
        plot=True,
        region=inversion_region,
        plot_type="pygmt",
        cmap="batlowW",
        diff_cmap="balance+h0",
        hist=True,
        inset=False,
        robust=True,
        verbose="q",
        cbar_label="m",
        shp_mask="../../plotting/RIS_outline.shp",
    )

In [ ]:
filtered_20k_MC_stats = []
for i, j in enumerate(MCstats):
    if i == 6:
        # low pass filter bed to remove some of the noise
        filtered = inv_utils.filter_grid(
            j,
            filter_width=20e3,
            filt_type="lowpass",
        )
        # mask bed outside of RIS
        filtered = utils.mask_from_shp(
            "../../plotting/RIS_outline.shp",
            xr_grid=filtered,
            masked=True,
            invert=False,
        )
        # merge back with unfiltered
        filtered = xr.where(
            filtered.isnull(),
            j,
            filtered,
        )
    else:
        filtered = inv_utils.filter_grid(
            j,
            filter_width=20e3,
            filt_type="lowpass",
        )

    filtered_20k_MC_stats.append(filtered)

    _ = utils.grd_compare(
        j,
        filtered,
        grid1_name="Unfiltered",
        grid2_name="Filtered",
        title="difference",
        plot=True,
        region=inversion_region,
        plot_type="pygmt",
        cmap="batlowW",
        diff_cmap="balance+h0",
        hist=True,
        inset=False,
        robust=True,
        verbose="q",
        cbar_label="m",
        shp_mask="../../plotting/RIS_outline.shp",
    )

In [ ]:
grids = [
    filtered_20k_MC_stats[6].rename("z"),
]
fig = maps.plot_3d(
    grids=grids,
    cmaps=["batlowW"],
    exaggeration=[20],
    shp_mask=gpd.read_file("../../plotting/RIS_outline.shp"),
    region=inversion_region,
    shading=True,
)

fig.show()

### Save to zarr

In [ ]:
# rename
names = [
    "stdev_gravity",
    "stdev_constraints",
    "stdev_densities",
    "stdev_starting_bed_tension",
    "stdev_regional_tension",
    "stdev_full",
    "mean_full",
]
renamed = []
for i, j in enumerate(filtered_20k_MC_stats):
    renamed.append(j.rename(names[i]))

merged = xr.merge(renamed)
merged

In [ ]:
enc = {x: {"compressor": zarr.Blosc()} for x in merged}
merged.to_zarr(
    "../../results/RIS_final_Monte_Carlo_inversion_results.zarr",
    encoding=enc,
    mode="w",
)

### Load from zarr

In [ ]:
filtered_MC_stats = xr.open_zarr(
    "../../results/RIS_final_Monte_Carlo_inversion_results.zarr"
)
MC_bed = filtered_MC_stats.mean_full
MC_uncert = filtered_MC_stats.stdev_full

filtered_MC_stats

In [ ]:
for var in list(filtered_MC_stats):
    da = utils.mask_from_shp(
        "../../plotting/RIS_outline.shp",
        xr_grid=filtered_MC_stats[var],
        masked=True,
        invert=False,
    ).rename("z")
    # df = vd.grid_to_table(da).dropna()
    # df.z.hist(bins=50)
    print(f"{var} mean: {np.nanmean(da.values)}m")
    print(f"{var} standard deviation: {np.nanstd(da.values)}m")

### Merge with Bedmachine

In [ ]:
info = utils.get_grid_info(bedmachine_bed_antarctica)
bedmachine_bed_antarctica = bedmachine_bed_antarctica.rename("bed")
bedmachine_bed_error_full = bedmachine_bed_error_full.rename("bed_error")

info

In [ ]:
# resample to match spacing and region of bedmachine
MC_bed_resample = fetch.resample_grid(
    MC_bed,
    spacing=info[0],
    registration=info[4],
    region=info[1],
    verbose="q",
).rename("bed")
MC_uncert_resample = fetch.resample_grid(
    MC_uncert,
    spacing=info[0],
    registration=info[4],
    region=info[1],
    verbose="q",
).rename("bed_error")

utils.get_grid_info(MC_bed_resample)

In [ ]:
MC_uncert_resample = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=MC_uncert_resample,
    masked=True,
    invert=False,
)
MC_uncert_resample.plot(vmin=100)

In [ ]:
# mask bedmachine data inside of shelf
bedmachine_bed_antarctica_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=bedmachine_bed_antarctica,
    masked=True,
    invert=True,
)
bedmachine_error_antarctica_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=bedmachine_bed_error_full,
    masked=True,
    invert=True,
)

In [ ]:
# align indexes of grids
aligned = xr.align(
    bedmachine_bed_antarctica_masked,
    bedmachine_bed_antarctica,
    MC_bed_resample,
    join="outer",
)
# merged inverted results within the ice shelf border
merged_inversion_bedmachine = xr.where(
    aligned[0].isnull(),
    aligned[2],
    aligned[1],
)

# align indexes of grids
aligned = xr.align(
    bedmachine_error_antarctica_masked,
    bedmachine_bed_error_full,
    MC_uncert_resample,
    join="outer",
)
# merged inverted results within the ice shelf border
merged_inversion_error_bedmachine = xr.where(
    aligned[0].isnull(),
    aligned[2],
    aligned[1],
)

utils.get_grid_info(merged_inversion_bedmachine)

In [ ]:
region = vd.pad_region(
    buffer_region,
    100e3,
)

RIS = merged_inversion_bedmachine.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
RIS.plot(robust=True)

In [ ]:
(merged_inversion_bedmachine - MC_bed).plot(robust=True)

In [ ]:
(merged_inversion_error_bedmachine - MC_uncert).plot(robust=True)

In [ ]:
region = vd.pad_region(
    buffer_region,
    500e3,
)

RIS = merged_inversion_bedmachine.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
(RIS - bedmachine_bed_antarctica).plot(robust=True)

In [ ]:
region = vd.pad_region(
    buffer_region,
    500e3,
)

RIS = merged_inversion_error_bedmachine.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
(RIS - bedmachine_bed_error_full).plot(robust=True)

In [ ]:
region = vd.pad_region(
    buffer_region,
    500e3,
)

RIS = merged_inversion_error_bedmachine.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
RIS.plot(robust=True)

### Save bedmachine merged results to zarr

In [ ]:
merged_with_bedmachine = (
    xr.merge(
        [
            merged_inversion_bedmachine,
            merged_inversion_error_bedmachine,
        ]
    )
    .load()
    .drop("mapping")
)
merged_with_bedmachine

In [ ]:
enc = {x: {"compressor": zarr.Blosc()} for x in merged_with_bedmachine}
merged_with_bedmachine.to_zarr(
    "../../results/antarctic_wide_results.zarr",
    encoding=enc,
    mode="w",
)

### Load from zarr

In [ ]:
merged_with_bedmachine = xr.open_zarr("../../results/antarctic_wide_results.zarr")
utils.get_grid_info(merged_with_bedmachine.bed)

### Save / load to netcdf

In [ ]:
merged_with_bedmachine.to_netcdf("../../results/antarctic_wide_results.nc")

In [ ]:
merged_with_bedmachine = xr.open_dataset("../../results/antarctic_wide_results.nc")
utils.get_grid_info(merged_with_bedmachine.bed)

In [ ]:
region = vd.pad_region(buffer_region, 500e3)

merged_with_bedmachine = merged_with_bedmachine.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
merged_with_bedmachine.bed.plot()

In [ ]:
region = vd.pad_region(buffer_region, 500e3)

merged_with_bedmachine = merged_with_bedmachine.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
merged_with_bedmachine.bed_error.plot()

In [ ]:
MC_bed_masked = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    gpd.read_file("../../plotting/RIS_outline.shp").buffer(-5e3),
    xr_grid=MC_bed,
    masked=True,
    invert=False,
)
print(f"Mean RIS bed depth: {np.nanmean(MC_bed_masked)} m")
print(f"Min RIS bed depth: {np.nanmin(MC_bed_masked)} m")
print(f"Max RIS bed depth: {np.nanmax(MC_bed_masked)} m")

final_MC_uncert_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    # gpd.read_file("../plotting/RIS_outline.shp").buffer(-5e3),
    xr_grid=filtered_MC_stats.stdev_full,
    masked=True,
    invert=False,
)
print(f"Mean RIS bed uncert: {np.nanmean(final_MC_uncert_masked)} m")
print(f"Min RIS bed uncert: {np.nanmin(final_MC_uncert_masked)} m")
print(f"Max RIS bed uncert: {np.nanmax(final_MC_uncert_masked)} m")

In [ ]:
points = profiles.sample_grids(
    constraints[constraints.inside],
    MC_bed,
    name="sampled_bed",
    coord_names=["easting", "northing"],
)
points["dif"] = points.upward - points.sampled_bed
utils.RMSE(points.dif)

In [ ]:
bed = vd.grid_to_table(MC_bed_masked).dropna().mean_full
bed.describe()

In [ ]:
bed.plot.hist(bins=500)

# Levelling results

In [ ]:
grav.plot_levelling_convergence(
    rosetta_levelled,
    mistie_prefix="mistie_trend",
    # title = "Mistie convergence",
    # logy=True,
)

In [ ]:
# set region
region = vd.pad_region(
    vd.get_region((rosetta_levelled.easting, rosetta_levelled.northing)), 100e3
)

grav_grid_levelled = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
    .Gobs
)

fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=10,
    transparency=50,
)
# lims = [-30,30]
lims = utils.get_min_max(grav_grid_levelled, robust=True)
pygmt.makecpt(
    cmap="viridis",
    continuous=True,
    series=lims,
    truncate=[-0.8, 0.8],
    background="o",
    output="../../plotting/tmp.cpt",
)
fig = maps.plot_grd(
    grav_grid_levelled,
    region=region,
    coast=True,
    coast_version="measures-v2",
    cbar_label="Levelled gravity disturbance (mGal)",
    hist=True,
    # cpt_lims=lims,
    # cmap="../plotting/tmp.cpt",
    cmap="viridis",
    robust=True,
    fig_height=10,
    frame=["nESw", "xa", "ya"],
    fig=fig,
    origin_shift="no_shift",
)
# plot point gravity data
fig.plot(
    x=rosetta_levelled.easting,
    y=rosetta_levelled.northing,
    style="p",
    transparency=50,
)

fig.show()

# Input datasets
Un-gridded gravity data and constraint points

In [ ]:
# set region
region = vd.pad_region(
    vd.get_region((rosetta_levelled.easting, rosetta_levelled.northing)), 90e3
)

# plot imagery as background
fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=10,
    transparency=40,
    title="ROSETTA-Ice Survey",
)

# make colormap
pygmt.makecpt(
    # cmap="balance+h0",
    cmap="viridis",
    continuous=True,
    series=utils.get_min_max(
        rosetta_levelled.disturbance,
        robust=True,
    ),
    background="o",
    output="../../plotting/tmp.cpt",
)

# plot point gravity data
fig.plot(
    x=rosetta_levelled.easting,
    y=rosetta_levelled.northing,
    color=rosetta_levelled.disturbance,
    style="c1.5p",
    cmap="../../plotting/tmp.cpt",
    frame=["neSW", "xa250000", "ya250000"],
)

# add colorbar
maps.add_colorbar(
    fig,
    cmap="../../plotting/tmp.cpt",
    cbar_yoffset=0.6,
    cbar_label="gravity disturbance (mGal)",
)

# plot coastline and groundind line
maps.add_coast(
    fig,
    version="measures-v2",
)

# add location labels
fig.text(
    x=260e3,
    y=-700e3,
    text="Transantarctic Mountains",
    angle=-45,
    font="12p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=260000,
    y=-700e3,
    text="Transantarctic Mountains",
    angle=-45,
    font="12p,Helvetica,black",
    justify="TC",
)
fig.text(
    x=-400e3,
    y=-650e3,
    text="Siple Coast",
    angle=35,
    font="12p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=-400e3,
    y=-650e3,
    text="Siple Coast",
    angle=35,
    font="12p,Helvetica,black",
    justify="TC",
)
fig.text(
    x=0,
    y=-1400e3,
    text="Ross Sea",
    angle=0,
    font="12p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=0, y=-1400e3, text="Ross Sea", angle=0, font="12p,Helvetica,black", justify="TC"
)


fig.text(
    position="TL",
    text="a",
    fill="white",
    font="14p,Helvetica,black",
    offset="j.5/.2",
    clearance="+tO",
)

info = utils.set_proj(
    region,
    fig_height=10,
)
maps.add_scalebar(fig, region, info[1], position="n.2/.05")
maps.add_north_arrow(fig, region, info[1], position="n.43/.07")

lims = [-1200, -200]
pygmt.grd2cpt(
    grid=bedmap2_bed_full,
    cmap="batlowW",
    # cmap="rain",
    # reverse=True,
    limit=lims,
    nlevels=11,
    output="../../plotting/tmp.cpt",
)

# plot bedmap2 bed as background
fig = maps.plot_grd(
    bedmap2_bed_full,
    # cmap="batlowW",
    cmap="../../plotting/tmp.cpt",
    cpt_lims=lims,
    region=region,
    coast=True,
    coast_version="measures-v2",
    cbar_label="bed elevation (m)",
    cbar_yoffset=0.6,
    title="Bathymetry constraints",
    frame=["nESw", "xa250000", "ya250000"],
    fig=fig,
    origin_shift="xshift",
)

# plot imagery
fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=10,
    transparency=80,
    fig=fig,
    origin_shift="no_shift",
)

# plot data
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    fill=constraints[constraints.inside].upward,
    style="c.12c",
    pen=".4p,black",
    cmap="../../plotting/tmp.cpt",
)

width = utils.get_fig_width(fig)

maps.add_inset(
    fig,
    region=region,
    inset_pos="TL",
    inset_width=0.25,
    inset_offset=f"-{((width+.8)*.25)/2}c/0c",
    inset_box="+gwhite+p1p+r12p",
)

# plot location labels from .tsv file, with white shadow
fig.text(
    textfiles="../../plotting/bathy_fig_labels.tsv",
    angle=True,
    font="8p,Helvetica,black,-=2p,white",
    justify=True,
)
fig.text(
    textfiles="../../plotting/bathy_fig_labels.tsv",
    angle=True,
    font="8p,Helvetica,black",
    justify=True,
)

fig.text(
    position="TL",
    text="b",
    fill="white",
    font="14p,Helvetica,black",
    offset="j1.5/.2",
    clearance="+tO",
)

fig.show()

In [ ]:
points = profiles.sample_grids(
    constraints[constraints.inside],
    bedmap2_bed_full,
    "sampled_bed",
    coord_names=["easting", "northing"],
)
points["dif"] = points.upward - points.sampled_bed
rmse = utils.RMSE(points.dif)
rmse

# Starting bathymetry models

In [ ]:
# get RMSE difference at constraints
rmses = []
for i in combined_list:
    points = profiles.sample_grids(
        constraints[constraints.inside],
        i,
        "sampled_bed",
        coord_names=["easting", "northing"],
    )
    points["dif"] = points.upward - points.sampled_bed
    rmse = utils.RMSE(points.dif)
    rmses.append(rmse)
rmses

In [ ]:
stats.weighted_mean

In [ ]:
# set plot region
region = buffer_region
# region = utils.alter_region(buffer_region, n_shift=-20e3)[0]
# region = vd.pad_region(inversion_region, 50e3,)

# set color limits for grids
# lims = [utils.get_min_max(i, robust=True) for i in combined_list]
lims = [-1400, -200]
pygmt.grd2cpt(
    grid=stats.weighted_mean,
    cmap="batlowW",
    limit=lims,
    nlevels=25,
    output="../../plotting/tmp.cpt",
)

for i, g in enumerate(combined_list):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
        yshift_amount = 1
    elif i == 3:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = -2.04
        yshift_amount = -0.95
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1.02
        yshift_amount = 1

    fig = maps.plot_grd(
        g,
        fig_height=10,
        region=region,
        coast=True,
        coast_version="measures-v2",
        cmap="../../plotting/tmp.cpt",
        cpt_lims=lims,
        cbar_label="bed elevation (m)",
        # hist=True,
        # cbar_yoffset=1,
        # hist_bin_num=50,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
        fig=fig,
    )
    fig.grdcontour(
        grid=g,
        interval=200,
        annotation=None,
        transparency=50,
    )
    fig.grdcontour(
        grid=g,
        interval=200,
        annotation=("400", "f6p"),
        transparency=50,
        limit=lims,
    )

    # plot constraints
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        fill=constraints[constraints.inside].upward,
        style="c.10c",
        pen=".4p,black",
        cmap="../../plotting/tmp.cpt",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=f"Tension = {g.name[8:]}, RMSE: {int(rmses[i])}m",
        fill="white",
        font="14p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # plot subplot labels
    fig.text(
        position="TL",
        text=string.ascii_lowercase[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    # if i == 0:
    #     fig.text(
    #         x = region[0]-60e3,
    #         y = np.mean([region[2],region[3]]),
    #         justify="BC",
    #         text = "Bi-harmonic splines",
    #         fill = 'white',
    #         angle = 90,
    #         font = '20p,Helvetica,black',
    #         clearance = '+tO',
    #         no_clip=True,
    #     )
    # if i == 3:
    #     fig.text(
    #         x = region[0]-60e3,
    #         y = np.mean([region[2],region[3]]),
    #         justify="BC",
    #         text = "Minimum curvature",
    #         fill = 'white',
    #         angle = 90,
    #         font = '20p,Helvetica,black',
    #         clearance = '+tO',
    #         no_clip=True,
    #     )
fig.show()

In [ ]:
stdev_clip = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=stats.weighted_stdev,
    masked=True,
    invert=False,
)

lims2 = [0, 100]
pygmt.grd2cpt(
    grid=stdev_clip,
    cmap="inferno",
    reverse=True,
    limit=lims2,
    nlevels=22,
    output="../../plotting/tmp2.cpt",
)
# plot standard deviation of all grids
fig = maps.plot_grd(
    stdev_clip,
    fig_height=10,
    region=region,
    coast=True,
    coast_version="measures-v2",
    cmap="../../plotting/tmp2.cpt",
    cpt_lims=lims2,
    cbar_label="standard deviation (m)",
    hist=True,
    cbar_yoffset=1,
    hist_bin_num=50,
    origin_shift="both_shift",
    # xshift_amount = -1.4,
    xshift_amount=-2.04,
    yshift_amount=-0.95,
    fig=fig,
)
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.06c",
    fill="white",
    pen=".5p,black",
)
# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Weighted standard deviation",
    fill="white",
    font="14p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
# plot subplot labels
fig.text(
    position="TL",
    text="g",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)

# plot mean bathymetry model elevation
fig = maps.plot_grd(
    stats.weighted_mean,
    fig_height=10,
    region=region,
    coast=True,
    coast_version="measures-v2",
    cmap="../../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="elevation (m)",
    hist=True,
    cbar_yoffset=1,
    hist_bin_num=50,
    origin_shift="xshift",
    xshift_amount=1.02,
    fig=fig,
)
fig.grdcontour(
    grid=stats.weighted_mean,
    interval=200,
    annotation=None,
    transparency=50,
)
fig.grdcontour(
    grid=stats.weighted_mean,
    interval=200,
    annotation=("400", "f6p"),
    transparency=50,
    limit=lims,
)
# sample grid at constraints
points = profiles.sample_grids(
    constraints[constraints.inside],
    stats.weighted_mean,
    "sampled_bed",
    coord_names=["easting", "northing"],
)
points["dif"] = points.upward - points.sampled_bed
rmse = utils.RMSE(points.dif)
# plot constraints
# fig.plot(
#     x=constraints[constraints.inside].easting,
#     y=constraints[constraints.inside].northing,
#     style="c.06c",
#     fill="white",
#     pen=".5p,black",
# )
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    fill=constraints[constraints.inside].upward,
    style="c.10c",
    pen=".4p,black",
    cmap="../../plotting/tmp.cpt",
)

# plot titles
fig.text(
    position="TR",
    justify="TR",
    text=f"Weighted mean, RMSE: {int(rmse)}m",
    fill="white",
    font="14p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
# plot subplot labels
fig.text(
    position="TL",
    text="h",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)


# plot difference with BedMap2
bedmap2_diff_clip = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=bedmap2_bed_layer_spacing - stats.weighted_mean,
    masked=True,
    invert=False,
)

lims3 = [-200, 200]
pygmt.grd2cpt(
    grid=bedmap2_diff_clip,
    cmap="balance+h0",
    limit=lims3,
    nlevels=21,
    output="../../plotting/tmp3.cpt",
)

fig = maps.plot_grd(
    bedmap2_diff_clip,
    fig_height=10,
    region=region,
    coast=True,
    coast_version="measures-v2",
    cmap="../../plotting/tmp3.cpt",
    cpt_lims=lims3,
    cbar_label="difference (m)",
    hist=True,
    cbar_yoffset=1,
    hist_bin_num=50,
    origin_shift="xshift",
    xshift_amount=1.02,
    fig=fig,
)
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.06c",
    fill="white",
    pen=".5p,black",
)
# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Bedmap2 comparison",
    fill="white",
    font="14p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
# plot subplot labels
fig.text(
    position="TL",
    text="i",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.show()

# Constraints grid


In [ ]:
min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
    mindist=layer_spacing / np.sqrt(2),
)

min_dist = utils.mask_from_shp(
    gpd.read_file("../../plotting/RIS_outline.shp").buffer(-5e3),
    # "../plotting/RIS_outline.shp",
    xr_grid=min_dist,
    masked=True,
    invert=False,
)
fig = maps.plot_grd(
    min_dist,
    coast=True,
    coast_version="measures-v2",
    region=inversion_region,
    fig_height=10,
    title="Weighting grid",
    # title_font="12p,Helvetica-Bold",
    cmap="dense",
    hist=True,
    cbar_yoffset=1.5,
    cbar_unit_annot="km",
    cbar_scale=0.001,
    hist_bin_num=50,
)

fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="x0.12c",
    pen=".5p",
    label="Ice shelf constraints",
)
fig.plot(
    x=constraints[constraints.inside == False].easting,
    y=constraints[constraints.inside == False].northing,
    style="p",
    # pen='.5p',
    label="Outside constraints",
)

pygmt.grd2cpt(
    grid=starting_bed_prisms.weights,
    continuous=True,
    background="i",
    cmap="dense",
)
with pygmt.config(
    FONT_ANNOT="Helvetica-Oblique",
    MAP_ANNOT_OFFSET_PRIMARY="18p",
    MAP_TICK_LENGTH_PRIMARY="0p",
):
    maps.add_colorbar(fig=fig, cbar_yoffset=1.5)

fig.legend()
fig.show()

# Pick a profiles

In [ ]:
# lines = profiles.draw_lines()

In [ ]:
# # get clicked points
# df = utils.shapes_to_df(lines)
# # extract first and last
# a = (df.x.iloc[0], df.y.iloc[0])
# b = (df.x.iloc[-1], df.y.iloc[-1])

# # siple coast
# a, b = ((-514139.22589550324, -886012.3396426054),
# (-30871.591933855758, -595196.2984418197))

# # ice front to CIR
# a,b = ((48403.646330947464, -1387918.0291082778),
#  (-221877.26567443524, -613795.4844574957))

# # Roosevelt Island to TAM
# a,b = ((-489471.998996549, -1177190.3876565588),
#  (190464.00058142765, -777830.3659641005))

# # KIS to Ross Island
# a,b = ((-391311.08538905036, -713591.4759308188),
# (288280.6584588745, -1311607.5333427137))

# South of Roosevelt to TAM
# a,b = ((-571375.4234697601, -1051511.5422168665),
#  (132779.8923736067, -719735.4502564461))

# RI across CIR to TAM
a, b = (
    (-519891.1656344592, -1189239.5151540928),
    (-5922.676400296554, -607607.5280552685),
)

a, b

# Terrain mass effect

In [ ]:
grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

data_grids = [
    grav_grids.ice_surface_grav
    + grav_grids.water_surface_grav
    + grav_grids.starting_bed_grav,
    grav_grids.Gobs,
    grav_grids.misfit,
]

data_names = [
    "Terrain effects",
    "Disturbance",
    "Topo-free disturbance",
]

data_dict = profiles.make_data_dict(
    names=data_names,
    grids=data_grids,
    colors=[
        # "cyan",
        "red",
        "black",
        "blue",
        # "purple",
    ],
    axes=[0, 0, 0],
)

# layers for profiles plot
topo_grids = [
    bedmachine_surface_full,
    bedmachine_icebase_full,
    stats.weighted_mean,
    # bedmap2_bed_full,
    stats.weighted_mean + stats.weighted_stdev,
    stats.weighted_mean - stats.weighted_stdev,
]

topo_names = [
    "ice",
    "water",
    "starting bed",
    # None,# "bedmap2 bed",
    None,
    None,
    # "<math>+\sigma</math>",
    # "<math>-\sigma</math>",
]
layers_dict = profiles.make_data_dict(
    names=topo_names,
    grids=topo_grids,
    colors=[
        "lightskyblue",
        "royalblue",
        "saddlebrown",
        # "mediumseagreen",
        "lightbrown",
        "lightbrown",
    ],
)

In [ ]:
fig, df_layers, df_data = profiles.plot_profile(
    "points",
    start=a,
    stop=b,
    num=10000,
    data_height=4,
    fig_height=9,
    fig_width=15,
    data_dict=data_dict,
    data_legend_loc="jBL+jTL+o0c/.4c",
    data_buffer=0.1,
    data_frame=["neSW", "ya+lgravity (mGal)", "xag+lDistance (m)"],
    layers_dict=layers_dict,
    layers_legend=True,
    layer_buffer=0.2,
    layers_legend_loc="jTL",
    # layers_legend_box="+gwhite",
    layers_legend_scale=0.5,
    share_yaxis=True,
    layers_pen_thickness=[1.5, 1.5, 1.5, 0.6, 0.6],
    # layers_pen_color="black",
    # layers_pen_style=[None,None,None,".","."],
    layer_transparency=[40, 40, 30, 100, 100],
    layers_frame=["nesW", "ya+lelevation (m)", "xf"],
    start_label="A",
    end_label="A' ",
)
fig.show()

In [ ]:
# get grid of min dists to nearest constraints
min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
)

df_layers = profiles.sample_grids(df_layers, min_dist, name="min_dist")

df_layers["is_constraint"] = np.where(df_layers.min_dist <= 5e3, True, False)
print(len(df_layers[df_layers.is_constraint]))

In [ ]:
fig.plot(
    x=df_layers[df_layers.is_constraint].dist,
    y=df_layers[df_layers.is_constraint]["0"],
    # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
    style="c0.05c",
    fill="purple",
    pen="1p,purple",
    label="constraints",
    transparency=100,
)
fig.plot(
    x=df_layers[df_layers.is_constraint].dist,
    y=df_layers[df_layers.is_constraint]["0"],
    # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
    style="c0.01c",
    fill="purple",
    pen="1p,purple",
)
fig.legend(
    box=False,
    position="jTR",
)

fig.show()

In [ ]:
grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

grids = [
    grav_grids.ice_surface_grav
    + grav_grids.water_surface_grav
    + grav_grids.starting_bed_grav,
    grav_grids.Gobs,
    grav_grids.misfit,
]

titles = [
    "Terrain effects",
    "Disturbance",
    "Topo-free disturbance",
]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
        yshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1
        yshift_amount = 1

    if i == 2:
        cmap = "balance+h0"
    else:
        cmap = "viridis"

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=buffer_region,
        fig_height=10,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        region=buffer_region,
        coast=True,
        coast_version="measures-v2",
        cmap=cmap,
        cpt_lims=lims,
        # title=titles[i],
        cbar_label="mGal",
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.06c",
        fill="white",
        pen=".5p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        # plot profiles location, and endpoints on map
        fig.plot(
            vd.line_coordinates(a, b, size=100),
            pen="2p,black",
        )
        fig.text(
            x=a[0],
            y=a[1],
            text="A",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
        fig.text(
            x=b[0],
            y=b[1],
            text="A' ",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
fig.show()

# Regional separation

In [ ]:
grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

data_grids = [
    grav_grids.misfit,
    grav_grids.reg,
    grav_grids.res,
]

data_names = [
    "Topo-free disturbance",
    "Regional",
    "Residual",
]

data_dict = profiles.make_data_dict(
    names=data_names,
    grids=data_grids,
    colors=[
        # "cyan",
        "red",
        "black",
        "blue",
        # "purple",
    ],
    axes=[0, 0, 0],
)

# layers for profiles plot
topo_grids = [
    bedmachine_surface_full,
    bedmachine_icebase_full,
    stats.weighted_mean,
    # bedmap2_bed_full,
    stats.weighted_mean + stats.weighted_stdev,
    stats.weighted_mean - stats.weighted_stdev,
]

topo_names = [
    "ice",
    "water",
    "starting bed",
    # None,# "bedmap2 bed",
    None,
    None,
    # "<math>+\sigma</math>",
    # "<math>-\sigma</math>",
]
layers_dict = profiles.make_data_dict(
    names=topo_names,
    grids=topo_grids,
    colors=[
        "lightskyblue",
        "royalblue",
        "saddlebrown",
        # "mediumseagreen",
        "lightbrown",
        "lightbrown",
    ],
)

In [ ]:
fig, df_layers, df_data = profiles.plot_profile(
    "points",
    start=a,
    stop=b,
    num=10000,
    data_height=4,
    fig_height=9,
    fig_width=15,
    data_dict=data_dict,
    data_legend_loc="jBL+jTL+o0c/.4c",
    data_buffer=0.1,
    data_frame=["neSW", "ya+lgravity (mGal)", "xag+lDistance (m)"],
    layers_dict=layers_dict,
    layers_legend=True,
    layer_buffer=0.2,
    layers_legend_loc="jTL",
    # layers_legend_box="+gwhite",
    layers_legend_scale=0.5,
    share_yaxis=True,
    layers_pen_thickness=[1.5, 1.5, 1.5, 0.6, 0.6],
    # layers_pen_color="black",
    # layers_pen_style=[None,None,None,".","."],
    layer_transparency=[40, 40, 30, 100, 100],
    layers_frame=["nesW", "ya+lelevation (m)", "xf"],
    start_label="A",
    end_label="A' ",
)
fig.show()

In [ ]:
# get grid of min dists to nearest constraints
min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
)

df_layers = profiles.sample_grids(df_layers, min_dist, name="min_dist")

df_layers["is_constraint"] = np.where(df_layers.min_dist <= 5e3, True, False)
print(len(df_layers[df_layers.is_constraint]))

In [ ]:
fig.plot(
    x=df_layers[df_layers.is_constraint].dist,
    y=df_layers[df_layers.is_constraint]["0"],
    # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
    style="c0.05c",
    fill="purple",
    pen="1p,purple",
    label="constraints",
    transparency=100,
)
fig.plot(
    x=df_layers[df_layers.is_constraint].dist,
    y=df_layers[df_layers.is_constraint]["0"],
    # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
    style="c0.01c",
    fill="purple",
    pen="1p,purple",
)
fig.legend(
    box=False,
    position="jTR",
)

fig.show()

In [ ]:
grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

grids = [
    grav_grids.misfit,
    grav_grids.reg,
    grav_grids.res,
]

titles = [
    "Topo-free disturbance",
    "Regional",
    "Residual",
]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    # if i == 2:
    #     cmap = "balance+h0"
    # else:
    #     cmap = "viridis"
    cmap = "balance+h0"

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=buffer_region,
        fig_height=10,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        coast=True,
        coast_version="measures-v2",
        region=buffer_region,
        cmap=cmap,
        cpt_lims=lims,
        # title=titles[i],
        cbar_label="mGal",
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.06c",
        fill="white",
        pen=".5p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        # plot profiles location, and endpoints on map
        fig.plot(
            vd.line_coordinates(a, b, size=100),
            pen="2p,black",
        )
        fig.text(
            x=a[0],
            y=a[1],
            text="A",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
        fig.text(
            x=b[0],
            y=b[1],
            text="A' ",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
fig.show()

## Compare regional field to Tinto's density model

In [ ]:
density = fetch.ROSETTA_gravity(version="density")
density

In [ ]:
chain = vd.Chain(
    [
        ("reduce", vd.BlockReduce(np.mean, layer_spacing)),
        ("spline", vd.Spline()),
    ]
)
chain.fit((density.x, density.y), density.density_model)

In [ ]:
grid = chain.grid(
    region=inversion_region,
    spacing=layer_spacing,
    # dims=["latitude", "longitude"],
    data_names="density",
)
grid = vd.distance_mask(
    data_coordinates=(density.x, density.y),
    maxdist=20e3,
    grid=grid,
)
grid

In [ ]:
grid.density.plot(robust=True)

# Combined terrain and regional figures

In [ ]:
grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

data_grids = [
    grav_grids.Gobs,
    grav_grids.misfit,
    grav_grids.reg,
    grav_grids.res,
]

data_names = [
    "Disturbance",
    "Topo-free disturbance",
    "Regional",
    "Residual",
]

data_dict = profiles.make_data_dict(
    names=data_names,
    grids=data_grids,
    colors=[
        "black",
        "red",
        "blue",
        "limegreen",
    ],
    # axes = [0,0,1],
)

# layers for profiles plot
topo_grids = [
    bedmachine_surface_full,
    bedmachine_icebase_full,
    stats.weighted_mean,
    # bedmap2_bed_full,
    stats.weighted_mean + stats.weighted_stdev,
    stats.weighted_mean - stats.weighted_stdev,
]

topo_names = [
    "ice",
    "water",
    "starting bed",
    # None,# "bedmap2 bed",
    None,
    None,
    # "<math>+\sigma</math>",
    # "<math>-\sigma</math>",
]
layers_dict = profiles.make_data_dict(
    names=topo_names,
    grids=topo_grids,
    colors=[
        "lightskyblue",
        "royalblue",
        "saddlebrown",
        # "mediumseagreen",
        "lightbrown",
        "lightbrown",
    ],
)

In [ ]:
fig, df_layers, df_data = profiles.plot_profile(
    "points",
    start=a,
    stop=b,
    num=10000,
    data_height=4,
    fig_height=9,
    fig_width=15,
    data_dict=data_dict,
    data_legend_loc="jBL+o0c/-.1c",  # "jBL+jTL+o0c/.4c",
    data_buffer=0.1,
    data_frame=["neSW", "ya+lgravity (mGal)", "xag+lDistance (m)"],
    data_pen_style=[".", "-", None, "4_2:1p"],
    data_pen_thickness=1.5,
    share_yaxis=True,
    layers_dict=layers_dict,
    layers_legend=True,
    layer_buffer=0.2,
    layers_legend_loc="jTL",
    # layers_legend_box="+gwhite",
    layers_legend_scale=0.5,
    layers_pen_thickness=[1.5, 1.5, 1.5, 0.8, 0.8],
    # layers_pen_color="black",
    # layers_pen_style=[None,".","-","-."],
    layer_transparency=[40, 40, 30, 100, 100],
    layers_frame=["nesW", "ya+lelevation (m)", "xf"],
    start_label="A",
    end_label="A' ",
)
fig.show()

In [ ]:
# get grid of min dists to nearest constraints
min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
)

df_layers = profiles.sample_grids(df_layers, min_dist, name="min_dist")

df_layers["is_constraint"] = np.where(df_layers.min_dist <= 5e3, True, False)
print(len(df_layers[df_layers.is_constraint]))

In [ ]:
series = [1e3, 10e3]
pygmt.makecpt(
    cmap="dense",
    reverse=True,
    series=series,
    background=True,
)
fig.plot(
    x=df_layers.dist,
    y=df_layers["0"],
    # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
    style="c0.08c",
    fill=df_layers.min_dist,
    cmap=True,
    # pen="1p,purple",
)
fig.legend(
    box=False,
    position="jTR",
)
fig.colorbar(
    cmap=True,
    position="jMR+w2c/.3c+jML+v+o0.2c/0c",
    frame=["xa+lnearest constraint", "y+lkm"],
    scale=0.001,
)


fig.show()

In [ ]:
# fig.plot(
#     x=df_layers[df_layers.is_constraint].dist,
#     y=df_layers[df_layers.is_constraint]["0"],
#     # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
#     style="c0.05c",
#     fill="purple",
#     pen="1p,purple",
#     label="constraints",
#     transparency=100,
# )
# fig.plot(
#     x=df_layers[df_layers.is_constraint].dist,
#     y=df_layers[df_layers.is_constraint]["0"],
#     # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
#     style="c0.01c",
#     fill="purple",
#     pen="1p,purple",
# )
# fig.legend(
#     box=False,
#     position="jTR",
# )

# fig.show()

In [ ]:
vd.grid_to_table(grav_grids.Gobs)

In [ ]:
grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

grids = [
    grav_grids.Gobs,
    grav_grids.misfit,
    grav_grids.reg,
    grav_grids.res,
]
titles = [
    "Disturbance",
    "Topo-free disturbance",
    "Regional",
    "Residual",
]

# set region
df = vd.grid_to_table(grav_grids.Gobs)
region = vd.pad_region(vd.get_region((df.easting, df.northing)), 50e3)

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    # if (i == 0) | (i==3):
    #     cmap = "balance+h0"
    # else:
    #     cmap = "viridis"
    cmap = "balance+h0"

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=10,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        # title=titles[i],
        cbar_label="mGal",
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.06c",
        fill="white",
        pen=".5p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 3:
        # plot profiles location, and endpoints on map
        fig.plot(
            vd.line_coordinates(a, b, size=100),
            pen="2p,black",
        )
        fig.text(
            x=a[0],
            y=a[1],
            text="A",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
        fig.text(
            x=b[0],
            y=b[1],
            text="A' ",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
    if i == 2:
        info = utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.43/.07")

fig.show()

# Inversion results figure

In [ ]:
# get final forward gravity dataarray
cols = [s for s in list(grav_results) if "_forward_grav" in s]
final_forward = grav_results[cols[-1]]

# compute final topo free disturbance
final_topo_free_disturbance = (
    grav_results.partial_topo_corrected_disturbance - final_forward
)
# compute resulting gravity residual
grav_residual = final_topo_free_disturbance - grav_results.reg

# get final residual dataarray to show its the same as the computed
cols = [s for s in list(grav_results) if "_final_misfit" in s]
max_abs_dif = vd.maxabs(grav_residual - grav_results[cols[-1]])
print(f"max diff between computed residual and final residual: {max_abs_dif}")

In [ ]:
final_bed_ocean_draft = bedmachine_icebase_layer_spacing - final_bed
final_bed_ocean_draft = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=final_bed_ocean_draft,
    masked=True,
    invert=False,
)

In [ ]:
grids = [final_bed, final_bed_ocean_draft, grav_residual]
titles = [
    "Inverted bathymetry",
    "Ocean cavity",
    "Gravity residual",
]
cbar_labels = [
    "elevation (m)",
    "thickness (m)",
    "gravity (mGal)",
]

fig_height = 10

# set region
region = buffer_region

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )

    if i == 0:
        lims = [-1300, -300]
        pygmt.grd2cpt(
            grid=g,
            cmap="batlowW",
            limit=lims,
            nlevels=11,  # background=True,
            output="../../plotting/tmp.cpt",
        )
    elif i == 1:
        lims = [0, 900]
        pygmt.grd2cpt(
            grid=g,
            cmap="rain",
            limit=[0, lims[1]],
            nlevels=10,  # background=True,
            output="../../plotting/tmp.cpt",
        )
    elif i == 2:
        lims = (-vd.maxabs(lims), vd.maxabs(lims))
        pygmt.makecpt(
            cmap="balance+h0",
            continuous=True,
            series=lims,
            truncate=[-0.8, 0.8],
            background="o",
            output="../../plotting/tmp.cpt",
        )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap="../../plotting/tmp.cpt",
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.06c",
        fill="white",
        pen=".5p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i+2]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 1:
        # plot profiles location, and endpoints on map
        fig.plot(
            vd.line_coordinates(a, b, size=100),
            pen="2p,black",
        )
        fig.text(
            x=a[0],
            y=a[1],
            text="A",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )
        fig.text(
            x=b[0],
            y=b[1],
            text="A' ",
            fill="white",
            font="12p,Helvetica,black",
            justify="CM",
            clearance="+tO",
        )

        info = utils.set_proj(
            region,
            fig_height=fig_height,
        )
        maps.add_scalebar(
            fig,
            region,
            info[1],
            position="n.2/.05",
        )
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")

fig.show()

In [ ]:
# layers for profiles plot
topo_grids = [
    bedmachine_surface_full,
    bedmachine_icebase_full,
    stats.weighted_mean,
    final_bed,
]

topo_names = ["ice", "water", "starting bed", "inverted bed"]
layers_dict = profiles.make_data_dict(
    names=topo_names,
    grids=topo_grids,
    colors=[
        "lightskyblue",
        "royalblue",
        "saddlebrown",
        "lightbrown",
    ],
)

grav_grids = (
    df_anomalies[df_anomalies.test == False]
    .set_index(["northing", "easting"])
    .to_xarray()
)

data_grids = [
    grav_grids.res,
    final_bed - stats.weighted_mean,
]

data_names = [
    "Initial residual gravity",
    "Inversion bathymetry correction",
]

data_dict = profiles.make_data_dict(
    names=data_names,
    grids=data_grids,
    colors=[
        "blue",
        "red",
    ],
    axes=[0, 1],
)

In [ ]:
fig, df_layers, df_data = profiles.plot_profile(
    "points",
    start=a,
    stop=b,
    num=10000,
    data_height=4,
    fig_height=9,
    fig_width=15,
    layers_dict=layers_dict,
    layers_legend=True,
    layer_buffer=0.2,
    layers_legend_loc="jTL+o0c/-.2c",
    # layers_legend_box="+gwhite",
    layers_legend_scale=0.5,
    layers_pen_thickness=1.5,
    layer_transparency=[40, 40, 30, 100],
    layers_frame=["nesW", "ya+lelevation (m)", "xf"],
    share_yaxis=True,
    data_dict=data_dict,
    data_pen_thickness=1.5,
    data_legend_loc="jTL",
    data_buffer=0.1,
    data_frame=[
        ["nSW", "ya+lgravity (mGal)", "xag+lDistance (m)"],
        ["E", "ya+lelevation (m)"],
    ],
    start_label="A",
    end_label="A' ",
)
fig.show()

In [ ]:
# get grid of min dists to nearest constraints
min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
)

df_layers = profiles.sample_grids(df_layers, min_dist, name="min_dist")

df_layers["is_constraint"] = np.where(df_layers.min_dist <= 5e3, True, False)
print(len(df_layers[df_layers.is_constraint]))

In [ ]:
series = [1e3, 10e3]
pygmt.makecpt(
    cmap="dense",
    reverse=True,
    series=series,
    background=True,
)
fig.plot(
    x=df_layers.dist,
    y=df_layers["0"],
    # y=np.ones_like(df_layers[df_layers.is_constraint])*-1100,
    style="c0.08c",
    fill=df_layers.min_dist,
    cmap=True,
    # pen="1p,purple",
)
fig.legend(
    box=False,
    position="jTR",
)
fig.colorbar(
    cmap=True,
    position="jMR+w2c/.3c+jML+v+o0.2c/0c",
    frame=["xa+lnearest constraint", "y+lkm"],
    scale=0.001,
)


fig.show()

# Monte Carlo figures

## Bathymetry

In [ ]:
MC_bed_masked = utils.mask_from_shp(
    # "../../plotting/RIS_outline.shp",
    gpd.read_file("../../plotting/RIS_outline.shp").buffer(-20e3),
    xr_grid=MC_bed,
    masked=True,
    invert=False,
)
MC_bed_masked.plot(vmin=-1500, vmax=-1300)

In [ ]:
min_point = (
    vd.grid_to_table(MC_bed_masked).dropna().sort_values(by="mean_full").iloc[0:2]
)
min_point

In [ ]:
points = profiles.sample_grids(
    min_point,
    merged_with_bedmachine.bed_error,
    name="sampled_bed",
    coord_names=["easting", "northing"],
)
points

In [ ]:
maps.plot_grd(
    MC_bed_masked,
    points=min_point.rename(columns={"easting": "x", "northing": "y"}),
    frame=True,
)

In [ ]:
fig_height = 10

lims = [-1300, -300]
pygmt.grd2cpt(
    grid=MC_bed,
    cmap="batlowW",
    limit=lims,
    nlevels=11,  # background=True,
    output="../../plotting/tmp.cpt",
)

fig = maps.plot_grd(
    grid=MC_bed,
    fig_height=fig_height,
    coast=True,
    coast_version="measures-v2",
    region=buffer_region,
    cmap="../../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="elevation (m)",
    hist=True,
    cbar_yoffset=1,
)
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.06c",
    fill="white",
    pen=".5p,black",
)
fig.grdcontour(
    grid=MC_bed,
    # interval=300,
    interval=1400,
    annotation=None,
    # transparency=50,
    # resample=10e3,
    # cut="1C",
)
# fig.grdcontour(
#     grid=MC_bed,
#     interval=200,
#     annotation=("+-400"),# "f6p"),
#     transparency=50,
#     limit=lims,
#     # resample=10e3,
#     cut="1C",
# )

fig.show()

## Bathy and Ocean

In [ ]:
MC_bed_ocean_draft = bedmachine_icebase_full - merged_with_bedmachine.bed

MC_bed_ocean_draft_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=MC_bed_ocean_draft,
    masked=True,
    invert=False,
)
MC_bed_ocean_draft = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    fetch.groundingline(version="measures-v2"),
    xr_grid=MC_bed_ocean_draft,
    masked=True,
    invert=True,
)
print(f"Mean ocean draft: {np.nanmean(MC_bed_ocean_draft_masked)} m")
print(f"Mean RIS ocean draft: {np.nanmean(MC_bed_ocean_draft_masked)} m")
print(f"Min RIS ocean draft: {np.nanmin(MC_bed_ocean_draft_masked)} m")
print(f"Max RIS ocean draft: {np.nanmax(MC_bed_ocean_draft_masked)} m")

In [ ]:
masked = utils.mask_from_shp(
    # "../../plotting/RIS_outline.shp",
    gpd.read_file("../../plotting/RIS_outline.shp").buffer(-60e3),
    xr_grid=MC_bed_ocean_draft,
    masked=True,
    invert=False,
)
masked.plot(vmin=700, vmax=1100)

In [ ]:
min_point = (
    vd.grid_to_table(masked)
    .dropna()
    .sort_values(by="scalars", ascending=False)
    .iloc[0:10]
)
min_point

In [ ]:
points = profiles.sample_grids(
    min_point,
    merged_with_bedmachine.bed_error,
    name="sampled_bed",
    coord_names=["easting", "northing"],
)
points

In [ ]:
maps.plot_grd(
    MC_bed_ocean_draft,
    points=min_point.rename(columns={"easting": "x", "northing": "y"}),
    frame=True,
)

In [ ]:
grids = [
    MC_bed,
    MC_bed_ocean_draft,
]
titles = [
    "Inverted bathymetry",
    "Ocean draft",
]
cbar_labels = [
    "elevation (m)",
    "thickness (m)",
]

fig_height = 10

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )

    if i == 0:
        lims = [-1400, -400]
        pygmt.grd2cpt(
            grid=g,
            cmap="batlowW",
            limit=lims,
            nlevels=11,  # background=True,
            output="../../plotting/tmp.cpt",
        )
    elif i == 1:
        lims = [0, 1000]
        pygmt.grd2cpt(
            grid=g,
            cmap="rain",
            limit=[0, lims[1]],
            nlevels=15,  # background=True,
            output="../../plotting/tmp.cpt",
        )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=buffer_region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        region=buffer_region,
        cmap="../../plotting/tmp.cpt",
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.06c",
        fill="white",
        pen=".5p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i+2]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 1:
        info = utils.set_proj(
            buffer_region,
            fig_height=fig_height,
        )
        maps.add_scalebar(
            fig,
            buffer_region,
            info[1],
            position="n.2/.05",
        )


fig.show()

## Uncertainty results

In [ ]:
titles = [
    "Final inverted bathymetry",
    "Total uncertainty",
]
cbar_labels = [
    "elevation (m)",
    "standard deviation (m)",
]
grids = [
    merged_with_bedmachine.bed,
    # filtered_MC_stats.mean_full,
    merged_with_bedmachine.bed_error,
    # filtered_MC_stats.stdev_full,
]

region = utils.alter_region(buffer_region, n_shift=-40e3)[0]

KIS = pd.read_csv("/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_KIS.csv")
KIS = utils.latlon_to_epsg3031(KIS, input=["Longitude", "Latitude"])
df = KIS[KIS.Station != "KIS"]
KIS = df[df.Station.astype("int") >= 3000]

DD = pd.read_csv(
    "/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_DD_2feb22.csv"
)
DD = utils.latlon_to_epsg3031(DD, input=["Longitude", "Latitude"])
DD = DD[DD.Name.str.contains("L")]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    if i == 0:
        cmap = "batlowW"
        reverse_cpt = False
    else:
        cmap = ("inferno",)
        reverse_cpt = True
        # g = utils.mask_from_shp(
        #     shapefile="../plotting/RIS_outline.shp",
        #     xr_grid=g,
        #     masked=True,
        #     invert=False,
        # )

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )
    grd2cpt = True

    if i == 0:
        lims = [-1300, -300]
        grd2cpt = False

    # fig = maps.plot_grd(
    #     fetch.modis_moa(),
    #     image=True,
    #     cmap="gray",
    #     region=region,
    #     fig_height=10,
    #     transparency=50,
    #     fig=fig,
    #     origin_shift=origin_shift,
    #     xshift_amount=xshift_amount,
    # )
    if i == 0:
        frame = ["NesW", "xa250000", "ya250000+ap"]
    elif i == 1:
        frame = ["NEsw", "xa250000", "ya250000+ap"]

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        frame=frame,
        reverse_cpt=reverse_cpt,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        # origin_shift="no_shift",
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )
    if i == 0:
        fig.grdcontour(
            grid=g,
            interval=300,
            annotation=None,
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=100,
            annotation=None,
            pen=".2p,white",
            transparency=50,
            # resample=10e3,
            cut="1C",
        )

    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # if i == 0:
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.04c",
        fill="white",
        pen=".3p,black",
    )
    fig.plot(x=KIS.x, y=KIS.y, pen=".7p,cyan")
    fig.plot(x=DD.x, y=DD.y, pen=".7p,cyan")

    # if i == 1:
    #     fig.plot(
    #         x=constraints[constraints.inside].easting,
    #         y=constraints[constraints.inside].northing,
    #         style="c.04c",
    #         fill="white",
    #         pen=".3p,black",
    #         label="constraint points",
    #     )
    #     fig.plot(x = KIS.x, y=KIS.y, pen=".9p,cyan", label="KIS survey")
    #     fig.plot(x = DD.x, y=DD.y, pen=".9p,cyan", label="Disco Deep survey")
    #     # fig.legend(position="JBR+jBR+o0c/-.8c", box="+gwhite+p1p,black",)

    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        info = utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")


fig.show()

In [ ]:
titles = [
    "Final inverted bathymetry",
    "Total uncertainty",
]
cbar_labels = [
    "elevation (m)",
    "standard deviation (m)",
]
grids = [
    merged_with_bedmachine.bed,
    # filtered_MC_stats.mean_full,
    merged_with_bedmachine.bed_error,
    # filtered_MC_stats.stdev_full,
]

region = utils.alter_region(buffer_region, n_shift=-40e3)[0]

KIS = pd.read_csv("/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_KIS.csv")
KIS = utils.latlon_to_epsg3031(KIS, input=["Longitude", "Latitude"])
df = KIS[KIS.Station != "KIS"]
KIS = df[df.Station.astype("int") >= 3000]

DD = pd.read_csv(
    "/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_DD_2feb22.csv"
)
DD = utils.latlon_to_epsg3031(DD, input=["Longitude", "Latitude"])
DD = DD[DD.Name.str.contains("L")]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    if i == 0:
        cmap = "batlowW"
        reverse_cpt = False
    else:
        cmap = ("inferno",)
        reverse_cpt = True
        # g = utils.mask_from_shp(
        #     shapefile="../plotting/RIS_outline.shp",
        #     xr_grid=g,
        #     masked=True,
        #     invert=False,
        # )

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )
    grd2cpt = True

    if i == 0:
        lims = [-1300, -300]
        grd2cpt = False

    # fig = maps.plot_grd(
    #     fetch.modis_moa(),
    #     image=True,
    #     cmap="gray",
    #     region=region,
    #     fig_height=10,
    #     transparency=50,
    #     fig=fig,
    #     origin_shift=origin_shift,
    #     xshift_amount=xshift_amount,
    # )
    if i == 0:
        frame = ["NesW", "xa250000", "ya250000+ap"]
        fig = maps.plot_grd(
            grid=g,
            fig_height=10,
            coast=True,
            coast_version="measures-v2",
            region=region,
            cmap=cmap,
            cpt_lims=lims,
            cbar_label=cbar_labels[i],
            frame=frame,
            reverse_cpt=reverse_cpt,
            hist=True,
            cbar_yoffset=1,
            fig=fig,
            # origin_shift="no_shift",
            origin_shift=origin_shift,
            xshift_amount=xshift_amount,
        )
    elif i == 1:
        frame = ["NEsw", "xa250000", "ya250000+ap"]
        g = utils.mask_from_shp(
            shapefile="../../plotting/RIS_outline.shp",
            xr_grid=g,
            masked=True,
            invert=False,
        )
        fig = maps.plot_grd(
            fetch.modis_moa(),
            image=True,
            cmap="gray",
            region=region,
            fig_height=10,
            transparency=50,
            fig=fig,
            origin_shift=origin_shift,
            xshift_amount=xshift_amount,
        )
        fig = maps.plot_grd(
            grid=g,
            fig_height=10,
            coast=True,
            coast_version="measures-v2",
            region=region,
            cmap=cmap,
            cpt_lims=lims,
            cbar_label=cbar_labels[i],
            frame=frame,
            reverse_cpt=reverse_cpt,
            hist=True,
            cbar_yoffset=1,
            fig=fig,
            origin_shift="no_shift",
        )
    if i == 0:
        fig.grdcontour(
            grid=g,
            interval=300,
            annotation=None,
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=100,
            annotation=None,
            pen=".2p,white",
            transparency=50,
            # resample=10e3,
            cut="1C",
        )

    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # if i == 0:
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.04c",
        fill="white",
        pen=".3p,black",
    )
    fig.plot(x=KIS.x, y=KIS.y, pen=".7p,cyan")
    fig.plot(x=DD.x, y=DD.y, pen=".7p,cyan")

    # if i == 1:
    #     fig.plot(
    #         x=constraints[constraints.inside].easting,
    #         y=constraints[constraints.inside].northing,
    #         style="c.04c",
    #         fill="white",
    #         pen=".3p,black",
    #         label="constraint points",
    #     )
    #     fig.plot(x = KIS.x, y=KIS.y, pen=".9p,cyan", label="KIS survey")
    #     fig.plot(x = DD.x, y=DD.y, pen=".9p,cyan", label="Disco Deep survey")
    #     # fig.legend(position="JBR+jBR+o0c/-.8c", box="+gwhite+p1p,black",)

    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        info = utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")


fig.show()

## Sensitivity results

In [ ]:
titles = [
    "Uncertainty from gravity",
    "Uncertainty from constraints",
    "Uncertainty from density",
    "Uncertainty from bed gridding",
    "Uncertainty from regional separation",
    "Total uncertainty",
]
grids = [
    filtered_MC_stats.stdev_gravity,
    filtered_MC_stats.stdev_constraints,
    filtered_MC_stats.stdev_densities,
    filtered_MC_stats.stdev_starting_bed_tension,
    filtered_MC_stats.stdev_regional_tension,
    filtered_MC_stats.stdev_full,
]

region = utils.alter_region(buffer_region, n_shift=-50e3)[0]
fig_height = 10
for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 0.98
        yshift_amount = 0.98
    elif i == 3:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = -1.96
        yshift_amount = -0.98
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 0.98
        yshift_amount = 0.98

    cmap = ("inferno",)
    reverse_cpt = True
    g = utils.mask_from_shp(
        shapefile="../../plotting/RIS_outline.shp",
        xr_grid=g,
        masked=True,
        invert=False,
    )

    lims = utils.get_min_max(
        g,
        robust=True,
        shapefile="../../plotting/RIS_outline.shp",
    )
    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
    )
    cbar_label = (
        "uncertainty (m), "
        f"mean:{round(np.nanmean(g))}, "
        f"stdev:{round(np.nanstd(g))}"
    )
    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_label,
        cbar_font="12p,Helvetica,black",
        reverse_cpt=reverse_cpt,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    fig.grdcontour(
        grid=g,
        interval=100,
        annotation=None,
        pen=".2p,white",
        transparency=50,
        # resample=10e3,
        # cut="1C",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 2:
        fig.plot(
            x=constraints[constraints.inside].easting,
            y=constraints[constraints.inside].northing,
            style="c.06c",
            fill="white",
            pen=".5p,black",
        )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 1:
        # add location labels
        fig.text(
            x=220e3,
            y=-660e3,
            text="Transantarctic Mountains",
            angle=-40,
            font="14p,Helvetica,black,-=2p,white",
            justify="TC",
        )
        fig.text(
            x=220e3,
            y=-660e3,
            text="Transantarctic Mountains",
            angle=-40,
            font="14p,Helvetica,black",
            justify="TC",
        )
        fig.text(
            x=-400e3,
            y=-650e3,
            text="Siple Coast",
            angle=35,
            font="14p,Helvetica,black,-=2p,white",
            justify="TC",
        )
        fig.text(
            x=-400e3,
            y=-650e3,
            text="Siple Coast",
            angle=35,
            font="14p,Helvetica,black",
            justify="TC",
        )
        fig.text(
            x=-200e3,
            y=-1320e3,
            text="Ross Sea",
            angle=0,
            font="14p,Helvetica,black,-=2p,white",
            justify="TC",
        )
        fig.text(
            x=-200e3,
            y=-1320e3,
            text="Ross Sea",
            angle=0,
            font="14p,Helvetica,black",
            justify="TC",
        )
    if i == 4:
        info = utils.set_proj(
            region,
            fig_height=fig_height,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

In [ ]:
# data_grids = filtered_MC_stats[:-1]

# data_names = [
#     "Uncertainty from gravity",
#     "Uncertainty from  constraints",
#     "Uncertainty from density",
#     "Uncertainty from bed gridding",
#     "Uncertainty from regional separation",

#     "Total uncertainty",
# ]

# data_dict = profiles.make_data_dict(
#     names=data_names,
#     grids=data_grids,
#     colors=[
#         "black",
#         "red",
#         "blue",
#         "cyan",
#         "green",
#         "magenta",
#     ],
#     # axes = [0,0,1],
# )

# # layers for profiles plot
# topo_grids = [
#     bedmachine_surface_full,
#     bedmachine_icebase_full,
#     MC_bed+filtered_MC_stats[-2], # upper uncert
#     MC_bed,
#     MC_bed,
#     MC_bed-filtered_MC_stats[-2], # low uncert
#     bedmap2_bed_full,
#     bedmachine_bed_full,
# ]

# topo_names = [
#     "ice",
#     "water",
#     None,
#     "inverted bed",
#     None,
#     None,
#     None,
#     None,
# ]
# layers_dict = profiles.make_data_dict(
#     names=topo_names,
#     grids=topo_grids,
#     colors=[
#         "lightskyblue",
#         "royalblue",
#         "burlywood",
#         "tan4",
#         "burlywood",
#         "tan4",
#         "dimgray",
#         "dimgray",
#     ],
# )

# # get grid of min dists to nearest constraints
# min_dist = inv_utils.normalized_mindist(
#     constraints,
#     starting_bed_prisms,
# )

In [ ]:
# fig, df_layers, df_data = profiles.plot_profile(
#     "points",
#     start=a,
#     stop=b,
#     num=10000,

#     data_height=4,
#     fig_height=9,
#     fig_width=15,

#     data_dict=data_dict,
#     data_legend_loc="jTR+jTL",#"jBL+o0c/-.1c",
#     data_buffer=.05,
#     data_frame=["neSW", "ya+luncertainty (m)", "xag+lDistance (m)"],
#     # data_pen_style=["..","-","4_2:1p",None,],
#     share_yaxis=True,

#     layers_dict=layers_dict,
#     layers_frame=["nesW", "ya+lelevation (m)", "xf"],
#     layers_legend=True,
#     layers_legend_loc="jTR+jTL+o.02c/0c",
#     # layers_legend_box="+gwhite+c.2",
#     layers_legend_scale=.5,
#     layers_legend_font="13p,Helvetica,black",
#     layer_buffer=0.05,
#     layers_pen_thickness=[
#         1.2,
#         1.2,
#         .5,
#         1.2,
#         1.2,
#         .5,
#         .4,
#         .4,
#     ],
#     layers_pen_color=[
#         "lightskyblue",
#         "royalblue",
#         "burlywood",
#         "saddlebrown",
#         "saddlebrown",
#         "burlywood",
#         "whitesmoke",
#         "whitesmoke",
#     ],
#     layer_transparency=[
#         40,
#         40,
#         40,
#         0,
#         70,
#         0,
#         100,
#         100,
#     ],
#     layers_pen_style=[None,None,None,None,None,None,".","-"],#"4_2:.4p"],

#     start_label="A",
#     end_label="A' ",
# )

# # sample distance to nearest constraints
# df_layers = profiles.sample_grids(df_layers, min_dist, name="min_dist")
# df_layers["is_constraint"]=np.where(
#     df_layers.min_dist <=5e3, True, False)

# # plot constraints along ice surface
# fig.plot(
#     x=df_layers[df_layers.is_constraint].dist,
#     y=df_layers[df_layers.is_constraint]["0"],
#     style="c0.01c",
#     fill="purple",
#     pen="1p,purple",
# )

# fig.show()

# MC result vs single inversion

In [ ]:
_ = utils.grd_compare(
    MC_bed,
    final_bed,
    grid1_name="Monte Carlo inverted bed",
    grid2_name="Single inverted bed",
    title="difference",
    plot=True,
    region=inversion_region,
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    plot_type="pygmt",
    cmap="batlowW",
    diff_cmap="balance+h0",
    hist=True,
    inset=False,
    points_style="x.15c",
    points_pen="1p",
    # subplot_labels=True,
    robust=True,
    # robust=False,
    verbose="q",
    cbar_label="elevation (m)",
    shp_mask="../plotting/RIS_outline.shp",
)

# Series of profiles

In [ ]:
def plot_profile_locations():
    for i, p in enumerate(profiles):
        df = pd.read_csv(p)
        df = profiles.create_profile(
            "polyline",
            polyline=df,
            num=1000,
        )

        start_label = string.ascii_uppercase[i]
        end_label = f"{string.ascii_uppercase[i]}' "

        # plot profiles location, and endpoints on map
        map_units_100km = 100e3 / (float(info[0][3:]) / 100)
        map_units_100km
        fig.plot(
            x=df.x,
            y=df.y,
            pen=".8p,magenta",
            style=f"f{map_units_100km}c/.2c",  # ticks every 100km
            # style=f"qd{map_units_100km*2}c:+l200+ukm+f6p", # labels every 200km
        )
        fig.text(
            x=df.loc[df.dist.idxmin()].x,
            y=df.loc[df.dist.idxmin()].y,
            text=start_label,
            fill="white",
            font="9p,Helvetica,black",
            justify="CM",
            clearance="+tO",
            transparency=10,
        )
        fig.text(
            x=df.loc[df.dist.idxmax()].x,
            y=df.loc[df.dist.idxmax()].y,
            text=end_label,
            fill="white",
            font="9p,Helvetica,black",
            justify="CM",
            clearance="+tO",
            transparency=10,
        )

In [ ]:
lims = [-1300, -300]
pygmt.grd2cpt(
    grid=merged_with_bedmachine.bed,
    cmap="batlowW",
    # reverse=True,
    limit=lims,
    nlevels=11,
    output="../../plotting/tmp.cpt",
)

region = vd.pad_region(inversion_region, 40e3)

fig_height = 12
fig = maps.plot_grd(
    merged_with_bedmachine.bed,
    region=region,
    fig_height=fig_height,
    coast=True,
    coast_version="measures-v2",
    cmap="../../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="elevation (m)",
    # hist=True,
    # cbar_yoffset=1,
    # shading=True,
)
info = utils.set_proj(
    region,
    fig_height=fig_height,
)

profiles = [
    "../../plotting/calving_front_profile.csv",
    "../../plotting/RI_calving_front_to_CIR_profile.csv",
    "../../plotting/CIR_profile.csv",
    "../../plotting/up_nimrod_profile.csv",
    "../../plotting/up_byrd_profile.csv",
    "../../plotting/disco_deep_traverse_profile.csv",
    "....//plotting/groundingline_profile.csv",
]

plot_profile_locations()

fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.07c",
    fill="white",
    pen=".6p,black",
)

# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Inverted bathymetry",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.text(
    position="TL",
    text="a",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
# plot location labels from .tsv file, with white shadow
# fig.text(
#     textfiles="../../plotting/bathy_fig_labels.tsv",
#     angle=True,
#     font="8p,Helvetica,black,-=2p,white",
#     justify=True,
# )
# fig.text(
#     textfiles="../../plotting/bathy_fig_labels.tsv",
#     angle=True,
#     font="8p,Helvetica,black",
#     justify=True,
# )
# add location labels
fig.text(
    x=-220000,
    y=-640000,
    text="Crary Ice Rise",
    angle=-35,
    font="12p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=-220000,
    y=-640000,
    text="Crary Ice Rise",
    angle=-35,
    font="12p,Helvetica,black",
    justify="TC",
)
fig.text(
    x=-380000,
    y=-850000,
    text="Siple Dome",
    angle=-30,
    font="12p,Helvetica,black,-=2p,white",
    justify="TR",
)
fig.text(
    x=-380000,
    y=-850000,
    text="Siple Dome",
    angle=-30,
    font="12p,Helvetica,black",
    justify="TR",
)


# plot second figure with imagery in background
fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=fig_height,
    transparency=50,
    fig=fig,
    origin_shift="xshift",
)

lims = [0, 1000]
pygmt.grd2cpt(
    grid=MC_bed_ocean_draft,
    cmap="rain",
    limit=lims,
    nlevels=11,
    output="../../plotting/tmp.cpt",
)

fig = maps.plot_grd(
    MC_bed_ocean_draft,
    region=region,
    fig_height=fig_height,
    # cmap="rain",
    cmap="../../plotting/tmp.cpt",
    coast=True,
    coast_version="measures-v2",
    no_coast=True,
    cbar_label="thickness (m)",
    cpt_lims=lims,
    fig=fig,
    origin_shift="no_shift",
)
fig.grdcontour(
    grid=MC_bed_ocean_draft,
    interval=200,
    annotation=None,
    transparency=50,
    # resample=10e3,
    cut="1C",
)
plot_profile_locations()

# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Ocean cavity",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.text(
    position="TL",
    text="b",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)

fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.07c",
    fill="white",
    pen=".6p,black",
)
# add location labels
fig.text(
    x=260e3,
    y=-700e3,
    text="Transantarctic Mountains",
    angle=-45,
    font="14p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=260000,
    y=-700e3,
    text="Transantarctic Mountains",
    angle=-45,
    font="14p,Helvetica,black",
    justify="TC",
)
fig.text(
    x=-400e3,
    y=-650e3,
    text="Siple Coast",
    angle=35,
    font="14p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=-400e3,
    y=-650e3,
    text="Siple Coast",
    angle=35,
    font="14p,Helvetica,black",
    justify="TC",
)
fig.text(
    x=0,
    y=-1400e3,
    text="Ross Sea",
    angle=0,
    font="14p,Helvetica,black,-=2p,white",
    justify="TC",
)
fig.text(
    x=0,
    y=-1400e3,
    text="Ross Sea",
    angle=0,
    font="14p,Helvetica,black",
    justify="TC",
)
maps.add_scalebar(
    fig,
    region,
    info[1],
    position="n.2/.05",
    # scalebar_box="+gwhite",
)


fig.show()

In [ ]:
# layers for profiles plot
topo_grids = [
    bedmachine_surface_full,
    bedmachine_icebase_full,
    MC_bed,
    MC_bed + MC_uncert,  # upper uncert
    MC_bed - MC_uncert,  # low uncert
    bedmap2_bed_full,
    bedmachine_bed_full,
]

topo_names = [
    "ice",
    "water",
    "inverted bed",
    None,
    None,
    None,
    None,
]
layers_dict = profiles.make_data_dict(
    names=topo_names,
    grids=topo_grids,
    colors=[
        # "lightskyblue",
        # "royalblue",
        # "saddlebrown",
        # "peru",
        # "peru",
        # "gray",
        # "black",
        "lightskyblue",
        "royalblue",
        "lightbrown",
        "peru",
        "peru",
        "red2",
        "black",
    ],
)

# get grid of min dists to nearest constraints
min_dist = inv_utils.normalized_mindist(
    constraints,
    starting_bed_prisms,
)

In [ ]:
for i, p in enumerate(profiles):
    df = pd.read_csv(p)

    if i == 6:
        layers_legend = False
        fig_width = 24
        fig_height = 5
    elif i == 0:
        layers_legend = True
        fig_width = 14
        fig_height = 5
    else:
        layers_legend = False
        fig_width = 14
        fig_height = 5

    # set axis labels and ticks
    if i in [0, 2, 4, 6]:
        frame = ["neSW", "ya+lelevation (m)", "xa+ldistance (m)"]
    elif i in [1, 3, 5]:
        frame = ["nESw", "ya+lelevation (m)", "xa+ldistance (m)"]

    fig, df_layers, df_data = profiles.plot_profile(
        "polyline",
        polyline=df,
        num=10000,
        fig_height=fig_height,
        fig_width=fig_width,
        layers_dict=layers_dict,
        layers_frame=frame,
        layers_legend=layers_legend,
        layers_legend_loc="jTL+o0c/-2.5c",
        layers_legend_box="+gwhite+c.2",
        layers_legend_scale=1.5,
        layers_legend_font="13p,Helvetica,black",
        layer_buffer=0.05,
        layers_pen_thickness=[
            1.5,
            1.5,
            1.5,
            1,
            1,
            1.2,
            1.2,
        ],
        layer_transparency=[
            0,
            0,
            0,
            100,
            100,
            100,
            100,
        ],
        layers_pen_style=[None, None, None, None, None, None, "4_2"],
        start_label=string.ascii_uppercase[i],
        end_label=f"{string.ascii_uppercase[i]}'",
        start_end_label_position="B",
        start_label_justify="BL",
        end_label_justify="BR",
        start_label_offset="0c/0c",
        end_label_offset="0c/0c",
    )

    # Labels
    if i == 0:
        fig.text(
            x=150e3,
            y=300,
            text="Roosevelt Island",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
    elif i == 1:
        fig.text(
            x=500e3,
            y=20,
            text="Crary Ice Rise",
            font="12p,Helvetica,black",
            justify="RB",
            no_clip=True,
        )
    elif i == 2:
        fig.text(
            x=10e3,
            y=150,
            text="KIS",
            font="12p,Helvetica,black",
            justify="LB",
            no_clip=True,
        )
        fig.text(
            x=280e3,
            y=150,
            text="Crary Ice Rise",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
    elif i == 3:
        fig.text(
            x=330e3,
            y=150,
            text="Nimrod Gl.",
            font="12p,Helvetica,black",
            justify="RB",
            no_clip=True,
        )
    elif i == 4:
        fig.text(
            x=250e3,
            y=-350,
            text="Byrd Gl.",
            font="12p,Helvetica,black",
            justify="RB",
            no_clip=True,
        )
    elif i == 5:
        fig.text(
            x=150e3,
            y=200,
            text="Discovery Deep",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=270e3,
            y=200,
            text="Mulock Gl.",
            font="12p,Helvetica,black",
            justify="RB",
            no_clip=True,
        )
    elif i == 6:
        fig.text(
            x=10e3,
            y=200,
            text="LAB",
            font="12p,Helvetica,black",
            justify="LB",
            no_clip=True,
        )
        fig.text(
            x=400e3,
            y=200,
            text="BIS",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=750e3,
            y=200,
            text="KIS",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=1200e3,
            y=200,
            text="WIS",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=1480e3,
            y=200,
            text="Shackleton",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=1680e3,
            y=200,
            text="Nimrod",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=1900e3,
            y=200,
            text="Byrd",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=2100e3,
            y=200,
            text="Minna Bluff",
            font="12p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )

        fig.text(
            x=600e3,
            y=450,
            text="Siple Coast",
            font="15p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=1650e3,
            y=450,
            text="Transantarctic Mts.",
            font="15p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=0,
            y=450,
            text="Ross Sea",
            font="15p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )
        fig.text(
            x=2300e3,
            y=450,
            text="Ross Sea",
            font="15p,Helvetica,black",
            justify="CB",
            no_clip=True,
        )

    # sample distance to nearest constraints
    df_layers = profiles.sample_grids(df_layers, min_dist, name="min_dist")
    df_layers["is_constraint"] = np.where(df_layers.min_dist <= 5e3, True, False)

    # # plot constraints along ice surface
    # fig.plot(
    #     x=df_layers[df_layers.is_constraint].dist,
    #     y=df_layers[df_layers.is_constraint]["0"],
    #     style="c0.01c",
    #     fill="purple",
    #     pen="1p,purple",
    # )

    series = [1e3, 10e3]
    pygmt.makecpt(
        cmap="dense",
        reverse=True,
        series=series,
        background=True,
    )
    fig.plot(
        x=df_layers.dist,
        y=df_layers["0"],
        style="c0.08c",
        fill=df_layers.min_dist,
        cmap=True,
    )
    if i == 0:
        # replot lines at fully transparent to include them in legend
        fig.plot(
            x=df_layers.dist,
            y=df_layers["5"],
            pen=f"3p,{layers_dict['5']['color']}",
            label="Bedmap2",
            transparency=100,
        )
        fig.plot(
            x=df_layers.dist,
            y=df_layers["6"],
            pen=f"3p,{layers_dict['6']['color']},6_2",
            label="BedMachine",
            transparency=100,
        )
        fig.plot(
            x=df_layers.dist,
            y=df_layers["4"],
            pen=f"3p,{layers_dict['4']['color']}",
            label="Uncertainty",
            transparency=100,
        )
        with pygmt.config(
            FONT="13p,Helvetica,black",
        ):
            fig.legend(
                box=False,
                position="jTL+o4c/-2.5c",
                S=2,
            )
            fig.colorbar(
                cmap=True,
                position="jTC+w4c/.6c+jBC+h+o4.2c/1.8c",
                frame=["xa+lnearest constraint", "y+lkm"],
                scale=0.001,
            )
    fig.savefig(
        f"../../figures/profile_{string.ascii_uppercase[i]}.png",
        dpi=300,
    )
    fig.show()

# Misc plots

## Plot residual gravity

In [ ]:
grav_residual = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=grav_residual,
    masked=True,
    invert=False,
)

lims = utils.get_min_max(grav_residual, robust=True)

pygmt.makecpt(
    cmap="balance+h0",
    continuous=True,
    series=lims,
    truncate=[-0.8, 0.8],
    background="o",
    output="../../plotting/tmp.cpt",
)

fig = maps.plot_grd(
    grav_residual,
    fig_height=10,
    region=buffer_region,
    # cmap="balance+h0",
    # robust=True,
    cmap="../../plotting/tmp.cpt",
    cpt_lims=lims,
    coast=True,
    coast_version="measures-v2",
    points=constraints[constraints.inside].rename(
        columns={"easting": "x", "northing": "y"}
    ),
    points_style="x0.12c",
    points_pen=".8p",
    title="Gravity residuals",
    cbar_label="mGal",
    hist=True,
    cbar_yoffset=1.5,
)

fig.show()

## Compare with Bedmap2

In [ ]:
# masked_final_bed = utils.mask_from_shp(
#     "../../plotting/RIS_outline.shp",
#     xr_grid=merged_with_bedmachine.bed,
#     masked=True,
#     invert=False,
# )
# masked_bedmap_bed = utils.mask_from_shp(
#     "../../plotting/RIS_outline.shp",
#     xr_grid=bedmap2_bed_full,
#     masked=True,
#     invert=False,
# )
dif = bedmap2_bed_full - merged_with_bedmachine.bed
dif_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)

In [ ]:
titles = [
    "Bedmap2 bed",
    f"difference, RMSE: {int(utils.RMSE(dif_masked))} m",
    "Inverted bed",
]
cbar_labels = [
    "elevation (m)",
    "difference (m)",
    "elevation (m)",
]
grids = [
    bedmap2_bed_full,
    dif_masked,
    merged_with_bedmachine.bed,
]

lims = [-1300, -300]
pygmt.grd2cpt(
    grid=merged_with_bedmachine.bed,
    cmap="batlowW",
    limit=lims,
    nlevels=11,
    output="../../plotting/tmp.cpt",
)

region = utils.alter_region(buffer_region, n_shift=-40e3)[0]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    if i == 1:
        cmap = "balance+h0"
        lims = utils.get_min_max(
            g,
            robust=True,
        )
        lims = (-vd.maxabs(lims), vd.maxabs(lims))
    else:
        cmap = "../../plotting/tmp.cpt"
        lims = [-1300, -300]

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=10,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )
    if i == 0:
        frame = ["NesW", "xa250000", "ya250000"]
    elif i == 1:
        frame = ["NEsw", "xa250000", "ya250000"]

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        frame=frame,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    if i == 1:
        fig.grdcontour(
            grid=g,
            interval=100,
            annotation=None,
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=400,
            annotation=None,
            # pen=".2p,white",
            transparency=50,
            # resample=10e3,
            cut="1C",
        )

    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # if i == 0:
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.03c",
        fill="white",
        pen=".25p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        info = utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

## Compare with Bedmachine

In [ ]:
dif = bedmachine_bed_full - merged_with_bedmachine.bed
dif_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=dif,
    masked=True,
    invert=False,
)

In [ ]:
titles = [
    "Bedmachine bed",
    f"difference, RMSE: {int(utils.RMSE(dif_masked))} m",
    "Inverted bed",
]
cbar_labels = [
    "elevation (m)",
    "difference (m)",
    "elevation (m)",
]
grids = [
    bedmachine_bed_full,
    dif_masked,
    merged_with_bedmachine.bed,
]

lims = [-1300, -300]
pygmt.grd2cpt(
    grid=merged_with_bedmachine.bed,
    cmap="batlowW",
    limit=lims,
    nlevels=11,
    output="../../plotting/tmp.cpt",
)

region = utils.alter_region(buffer_region, n_shift=-40e3)[0]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    if i == 1:
        cmap = "balance+h0"
        lims = utils.get_min_max(
            g,
            robust=True,
        )
        lims = (-vd.maxabs(lims), vd.maxabs(lims))
    else:
        cmap = "../../plotting/tmp.cpt"
        lims = [-1300, -300]

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=10,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )
    if i == 0:
        frame = ["NesW", "xa250000", "ya250000"]
    elif i == 1:
        frame = ["NEsw", "xa250000", "ya250000"]

    fig = maps.plot_grd(
        grid=g,
        fig_height=10,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        frame=frame,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    if i == 1:
        fig.grdcontour(
            grid=g,
            interval=100,
            annotation=None,
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=400,
            annotation=None,
            # pen=".2p,white",
            transparency=50,
            # resample=10e3,
            cut="1C",
        )

    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # if i == 0:
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.03c",
        fill="white",
        pen=".25p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        info = utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

## Compare with both

In [ ]:
bedmap2_dif = bedmap2_bed_full - merged_with_bedmachine.bed
bedmap2_dif_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=bedmap2_dif,
    masked=True,
    invert=False,
)
bedmachine_dif = bedmachine_bed_full - merged_with_bedmachine.bed
bedmachine_dif_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=bedmachine_dif,
    masked=True,
    invert=False,
)

bedmachine_bedmap2_dif = bedmap2_bed_full - bedmachine_bed_full
bedmachine_bedmap2_dif_masked = utils.mask_from_shp(
    "../../plotting/RIS_outline.shp",
    xr_grid=bedmachine_bedmap2_dif,
    masked=True,
    invert=False,
)

In [ ]:
points = profiles.sample_grids(
    constraints[constraints.inside],
    merged_with_bedmachine.bed,
    name="sampled_bed",
    coord_names=["easting", "northing"],
)
points["dif"] = points.upward - points.sampled_bed
inverted_const_rmse = int(utils.RMSE(points.dif))
print(f"inverted bed RMS at constraints: {inverted_const_rmse}")


points = profiles.sample_grids(
    constraints[constraints.inside],
    bedmachine_bed_full,
    name="sampled_bed",
    coord_names=["easting", "northing"],
)
points["dif"] = points.upward - points.sampled_bed
bedmachine_const_rmse = int(utils.RMSE(points.dif))
print(f"bedmachine RMS at constraints: {bedmachine_const_rmse}")


points = profiles.sample_grids(
    constraints[constraints.inside],
    bedmap2_bed_full,
    name="sampled_bed",
    coord_names=["easting", "northing"],
)
points["dif"] = points.upward - points.sampled_bed
bedmap2_const_rmse = int(utils.RMSE(points.dif))
print(f"bedmap2 RMS at constraints: {bedmap2_const_rmse}")

In [ ]:
titles = [
    "Bedmap2 bathymetry",
    "Inverted bathymetry",
    "BedMachine bathymetry",
    "Bedmap2 - Inverted",
    "Bedmap2 - BedMachine",
    "BedMachine - Inverted",
]
subtitles = [
    f"constraints RMSE: {bedmap2_const_rmse} m",
    f"constraints RMSE: {inverted_const_rmse} m",
    f"constraints RMSE: {bedmachine_const_rmse} m",
    f"RMSE: {int(utils.RMSE(bedmap2_dif_masked))} m",
    f"RMSE: {int(utils.RMSE(bedmachine_bedmap2_dif))} m",
    f"RMSE: {int(utils.RMSE(bedmachine_dif_masked))} m",
]
cbar_labels = [
    "elevation (m)",
    "elevation (m)",
    "elevation (m)",
    "difference (m)",
    "difference (m)",
    "difference (m)",
]
grids = [
    bedmap2_bed_full,
    merged_with_bedmachine.bed,
    bedmachine_bed_full,
    bedmap2_dif_masked,
    bedmachine_bedmap2_dif_masked,
    bedmachine_dif_masked,
]
lims = [-1300, -300]
pygmt.grd2cpt(
    grid=merged_with_bedmachine.bed,
    cmap="batlowW",
    limit=lims,
    nlevels=11,
    output="../../plotting/tmp.cpt",
)

region = utils.alter_region(buffer_region, n_shift=-40e3)[0]
fig_height = 10
for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
        yshift_amount = 1
    elif i == 3:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = -2
        yshift_amount = -1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1
        yshift_amount = 1

    # if (i==0) | (i==3) | (1==4):
    if i > 2:
        cmap = "balance+h0"
        # lims = utils.get_min_max(g, robust=True,)
        # lims = (-vd.maxabs(lims),vd.maxabs(lims))
        lims = [-300, 300]
    else:
        cmap = "../../plotting/tmp.cpt"
        lims = [-1300, -300]

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
    )
    # if (i==0) | (i==3):
    #     frame=["NWsw", "xa250000", "ya250000"]
    frame = None

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        frame=frame,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    if i > 2:
        fig.grdcontour(
            grid=g,
            interval=100,
            annotation=None,
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=400,
            annotation=None,
            # pen=".2p,white",
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    # plot place names
    if i == 4:
        # plot location labels from .tsv file, with white shadow
        fig.text(
            textfiles="../../plotting/fig_labels_no_bathy_features.tsv",
            angle=True,
            font="9p,Helvetica,black,-=2p,white",
            justify=True,
        )
        fig.text(
            textfiles="../../plotting/fig_labels_no_bathy_features.tsv",
            angle=True,
            font="9p,Helvetica,black",
            justify=True,
        )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # plot subtitles
    fig.text(
        position="TR",
        justify="TR",
        text=subtitles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.9",
        clearance="+tO",
    )

    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.03c",
        fill="white",
        pen=".25p,black",
    )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="20p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 4:
        info = utils.set_proj(
            region,
            fig_height=fig_height,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

## Bedmap2 and Bedmachine plots

## Compare with starting

## Plot upward derivative

In [ ]:
ds = inv_result[0].set_index(["northing", "easting"]).to_xarray()
cols = [s for s in inv_result[0].columns.to_list() if "_layer" in s]
final_bed = ds[cols[-1]]

up_deriv = np.abs(inv_utils.filter_grid(final_bed, filt_type="up_deriv"))

pygmt.grd2cpt(
    grid=final_bed,
    cmap="rain",
    reverse=True,
    limit="-1500/0",
    nlevels=16,  # background=True,
    output="plotting/RIS_bathy.cpt",
)
fig = maps.plot_grd(
    final_bed,
    fig_height=10,
    coast=True,
    coast_version="measures-v2",
    title="Inverted bed (verde Spline)",
    # title="Inverted bed (GMT surface)",
    # region=regions.ross_ice_shelf,
    cmap="plotting/RIS_bathy.cpt",  # "rain",
    # reverse_cpt=True,
    # grd2cpt=True,
    cpt_lims=[-1500, 0],
    cbar_label="elevation (m)",
    hist=True,
    cbar_yoffset=1.5,
)
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.06c",
    fill="white",
    pen=".5p,black",
)
info = utils.set_proj(
    regions.ross_ice_shelf,
    fig_height=10,
)
maps.add_scalebar(fig, regions.ross_ice_shelf, info[1], position="n.2/-.05")

fig = maps.plot_grd(
    up_deriv,
    coast=True,
    coast_version="measures-v2",
    # region=regions.ross_ice_shelf,
    cmap="matter",
    grd2cpt=True,
    hist=True,
    cbar_yoffset=1.5,
    cbar_label="Upward derivative (absolute value)",
    robust=True,
    cpt_lims=[0, 0.03],
    fig=fig,
    origin_shift="xshift",
)
fig.plot(
    x=constraints[constraints.inside].easting,
    y=constraints[constraints.inside].northing,
    style="c.06c",
    fill="white",
    pen=".5p,black",
)
fig.show()

## Plot ocean drafts

In [ ]:
bedmap2_ocean_draft = bedmap2_icebase_full - bedmap2_bed_full
bedmachine_ocean_draft = bedmachine_icebase_full - bedmachine_bed_full

bedmap2_ocean_draft = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    fetch.groundingline(version="measures-v2"),
    xr_grid=bedmap2_ocean_draft,
    masked=True,
    invert=True,
)
bedmachine_ocean_draft = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    fetch.groundingline(version="measures-v2"),
    xr_grid=bedmachine_ocean_draft,
    masked=True,
    invert=True,
)

In [ ]:
# region = utils.alter_region(buffer_region, n_shift=-40e3)[0]
region = buffer_region
fig_height = 10

lims = [0, 1000]
pygmt.grd2cpt(
    grid=MC_bed_ocean_draft,
    cmap="rain",
    limit=lims,
    nlevels=11,  # background=True,
    output="../plotting/tmp.cpt",
)

grids = [
    MC_bed_ocean_draft,
    bedmap2_ocean_draft,
    bedmachine_ocean_draft,
]
titles = [
    "Inversion results ocean draft",
    "Bedmap2 ocean draft",
    "Bedmachine ocean draft",
]

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
    )
    if i == 0:
        frame = ["NesW", "xa250000", "ya250000"]
    elif i == 2:
        frame = ["NEsw", "xa250000", "ya250000"]
    else:
        frame = None

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        no_coast=True,
        region=region,
        cmap="../plotting/tmp.cpt",
        cpt_lims=lims,
        cbar_label="thickness (m)",
        frame=frame,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )

    fig.grdcontour(
        grid=g,
        interval=200,
        annotation=None,
        transparency=50,
        # resample=10e3,
        cut="1C",
    )

    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # fig.plot(
    #     x=constraints[constraints.inside].easting,
    #     y=constraints[constraints.inside].northing,
    #     style="c.03c",
    #     fill="white",
    #     pen=".25p,black",
    # )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )

    if i == 1:
        info = utils.set_proj(
            region,
            fig_height=10,
        )
        maps.add_scalebar(
            fig, region, info[1], scalebar_box="+gwhite", position="n.2/.07"
        )
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

### Compare ocean draft with past models

In [ ]:
bedmap2_ocean_draft = bedmap2_icebase_full - bedmap2_bed_full
bedmachine_ocean_draft = bedmachine_icebase_full - bedmachine_bed_full
MC_bed_ocean_draft = bedmachine_icebase_full - merged_with_bedmachine.bed

bedmap2_ocean_draft = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    fetch.groundingline(version="measures-v2"),
    xr_grid=bedmap2_ocean_draft,
    masked=True,
    invert=True,
)
bedmachine_ocean_draft = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    fetch.groundingline(version="measures-v2"),
    xr_grid=bedmachine_ocean_draft,
    masked=True,
    invert=True,
)
MC_bed_ocean_draft = utils.mask_from_shp(
    # "../plotting/RIS_outline.shp",
    fetch.groundingline(version="measures-v2"),
    xr_grid=MC_bed_ocean_draft,
    masked=True,
    invert=True,
)


bedmap2_ocean_dif = MC_bed_ocean_draft - bedmap2_ocean_draft
bedmap2_ocean_dif_masked = utils.mask_from_shp(
    "../plotting/RIS_outline.shp",
    xr_grid=bedmap2_ocean_dif,
    masked=True,
    invert=False,
)
bedmachine_ocean_dif = MC_bed_ocean_draft - bedmachine_ocean_draft
bedmachine_ocean_dif_masked = utils.mask_from_shp(
    "../plotting/RIS_outline.shp",
    xr_grid=bedmachine_ocean_dif,
    masked=True,
    invert=False,
)

In [ ]:
titles = [
    f"difference, RMSE: {int(utils.RMSE(bedmap2_ocean_dif_masked))} m",
    "Ocean cavity",
    "Ocean cavity from inversion",
    f"difference, RMSE: {int(utils.RMSE(bedmachine_ocean_dif_masked))} m",
    "Ocean cavity",
]
cbar_labels = [
    "difference (m)",
    "thickness (m)",
    "thickness (m)",
    "difference (m)",
    "thickness (m)",
]
grids = [
    bedmap2_ocean_dif_masked,
    bedmap2_ocean_draft,
    MC_bed_ocean_draft,
    bedmachine_ocean_dif_masked,
    bedmachine_ocean_draft,
]

lims = [0, 1000]
pygmt.grd2cpt(
    grid=merged_with_bedmachine.bed,
    cmap="rain",
    limit=lims,
    nlevels=21,
    output="../plotting/tmp.cpt",
)

region = utils.alter_region(buffer_region, n_shift=-40e3)[0]
fig_height = 10
for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
        yshift_amount = 1
    elif i == 2:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = -2.1
        yshift_amount = -0.5
    elif i == 3:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = 1.1
        yshift_amount = -0.5
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1
        yshift_amount = 1

    if (i == 0) | (i == 3):
        cmap = "balance+h0"
        # lims = utils.get_min_max(g, robust=True,)
        # lims = (-vd.maxabs(lims),vd.maxabs(lims))
        lims = [-300, 300]
    else:
        cmap = "../plotting/tmp.cpt"
        lims = [0, 1000]

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
    )
    # if (i==0) | (i==3):
    #     frame=["NWsw", "xa250000", "ya250000"]
    # elif (i==1) | (i==4):
    #     frame=None
    # elif (i==2) | (i==6):
    #     frame=["NWsw", "xa250000", "ya250000"]
    frame = None

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        no_coast=True,
        region=region,
        cmap=cmap,
        cpt_lims=lims,
        cbar_label=cbar_labels[i],
        frame=frame,
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    if (i == 0) | (i == 3):
        fig.grdcontour(
            grid=g,
            interval=100,
            annotation=None,
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
    else:
        fig.grdcontour(
            grid=g,
            interval=200,
            annotation=None,
            # pen=".2p,white",
            transparency=50,
            # resample=10e3,
            cut="1C",
        )
        fig.grdcontour(
            grid=g,
            interval=20,
            # annotation=("+10"),
            limit=[18, 22],
            pen=".2p,cyan",
            # transparency=0,
            # resample=10e3,
            # cut="1C",
        )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    # if i == 0:
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.03c",
        fill="white",
        pen=".25p,black",
    )
    # plot place names
    if i == 3:
        # plot location labels from .tsv file, with white shadow
        fig.text(
            textfiles="../plotting/fig_labels_no_bathy_features.tsv",
            angle=True,
            font="9p,Helvetica,black,-=2p,white",
            justify=True,
        )
        fig.text(
            textfiles="../plotting/fig_labels_no_bathy_features.tsv",
            angle=True,
            font="9p,Helvetica,black",
            justify=True,
        )
    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 0:
        fig.text(
            x=region[0] - 60e3,
            y=np.mean([region[2], region[3]]),
            justify="BC",
            text="Bedmap2",
            fill="white",
            angle=90,
            font="24p,Helvetica,black",
            clearance="+tO",
            no_clip=True,
        )
    if i == 3:
        fig.text(
            x=region[0] - 60e3,
            y=np.mean([region[2], region[3]]),
            justify="BC",
            text="Bedmachine",
            fill="white",
            angle=90,
            font="24p,Helvetica,black",
            clearance="+tO",
            no_clip=True,
        )
    if i == 3:
        info = utils.set_proj(
            region,
            fig_height=fig_height,
        )
        maps.add_scalebar(fig, region, info[1], position="n.2/.05")
        maps.add_north_arrow(fig, region, info[1], position="n.45/.07")
fig.show()

## Basal Melt

In [ ]:
# from Adusumilli et al. 2020
adusumilli_basal_melt = fetch.basal_melt()

# from Das et al. 2020
das_melt_df = pd.read_csv(
    "../data/BasalMelt_Das_JGR2020.txt",
    header=None,
    delim_whitespace=True,
    names=["lon", "lat", "melt_rate"],
)

das_melt_df["melt_rate"] = das_melt_df.melt_rate * -1

# re-project to polar stereographic
from pyproj import Transformer

transformer = Transformer.from_crs("epsg:4326", "epsg:3031")
das_melt_df["easting"], das_melt_df["northing"] = transformer.transform(
    das_melt_df.lat.tolist(), das_melt_df.lon.tolist()
)

# block reduce to help with plotting
# melt_df = utils.block_reduce(
#     melt_df,
#     np.median,
#     spacing=10e3,
#     input_coord_names=["easting", "northing"],
#     input_data_names=["melt_rate"],
#     center_coordinates=False,
#     adjust="spacing",
#     drop_coords=False,
# )
das_melt_df.describe()

In [ ]:
chain = vd.Chain(
    [
        ("reduce", vd.BlockReduce(np.mean, 2e3)),
        ("spline", vd.SplineCV()),
    ]
)
chain.fit((das_melt_df.easting, das_melt_df.northing), das_melt_df.melt_rate)

In [ ]:
grid = chain.grid(
    region=inversion_region,
    spacing=layer_spacing,
    # dims=["latitude", "longitude"],
    data_names="melt_rate",
)
das_melt_grid = vd.distance_mask(
    data_coordinates=(das_melt_df.easting, das_melt_df.northing),
    maxdist=20e3,
    grid=grid,
).melt_rate
das_melt_grid.plot(robust=True)

In [ ]:
# das_melt_grid_filt = inv_utils.filter_grid(
#     das_melt_grid,
#     filter_width=50e3,
#     filt_type="lowpass",
#     change_spacing=False,
# )
# das_melt_grid_filt.plot(robust=True)

In [ ]:
region = vd.pad_region(
    buffer_region,
    100e3,
)

adusumilli_basal_melt = adusumilli_basal_melt.sel(
    easting=slice(region[0], region[1]),
    northing=slice(region[2], region[3]),
)
adusumilli_basal_melt_filt = inv_utils.filter_grid(
    adusumilli_basal_melt,
    filter_width=50e3,
    filt_type="lowpass",
    change_spacing=False,
)
adusumilli_basal_melt_filt.plot(robust=True)

In [ ]:
(das_melt_grid - adusumilli_basal_melt_filt).plot(robust=True)

In [ ]:
region = list(inversion_region)
region[2] = region[2] - 100e3

fig_height = 10

fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=fig_height,
    transparency=50,
)
# make colormap
lims = [-1, 1]
pygmt.grd2cpt(
    grid=das_melt_grid,
    cmap="balance+h0",
    limit=lims,
    nlevels=11,
    truncate=[-0.95, 0.95],
    background="o",
    output="../plotting/tmp.cpt",
)
fig = maps.plot_grd(
    grid=das_melt_grid,
    fig_height=fig_height,
    region=region,
    cmap="../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="basal melt rate (m/yr)",
    hist=True,
    cbar_yoffset=1,
    fig=fig,
    origin_shift="no_shift",
    transparency=100,
)
# plot ocean draft
lims2 = [0, 1000]
pygmt.grd2cpt(
    grid=MC_bed_ocean_draft,
    cmap="algae",
    limit=lims2,
    nlevels=11,
    output="../plotting/tmp2.cpt",
)
fig = maps.plot_grd(
    grid=MC_bed_ocean_draft,
    fig_height=fig_height,
    coast=True,
    coast_version="measures-v2",
    no_coast=True,
    region=region,
    cmap="../plotting/tmp2.cpt",
    cpt_lims=lims2,
    cbar_label="water column thickness (m)",
    colorbar=False,
    # hist=True,
    # cbar_yoffset=3,
    fig=fig,
    origin_shift="no_shift",
)

fig.grdcontour(
    grid=das_melt_grid,
    interval="../plotting/tmp.cpt",
    annotation="n",
    pen="1p,+cl",
)

# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Airborne radar",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.text(
    position="TL",
    text="a",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)

fig_width = utils.get_fig_width(fig)


fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=fig_height,
    transparency=50,
    fig=fig,
    origin_shift="xshift",
)
# make colormap
lims = [-1, 1]
pygmt.grd2cpt(
    grid=adusumilli_basal_melt_filt,
    cmap="balance+h0",
    limit=lims,
    nlevels=11,
    truncate=[-0.95, 0.95],
    background="o",
    output="../plotting/tmp.cpt",
)
fig = maps.plot_grd(
    grid=adusumilli_basal_melt_filt,
    fig_height=fig_height,
    region=region,
    cmap="../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="basal melt rate (m/yr)",
    hist=True,
    cbar_yoffset=1,
    fig=fig,
    origin_shift="no_shift",
    transparency=100,
)
fig = maps.plot_grd(
    grid=MC_bed_ocean_draft,
    fig_height=fig_height,
    coast=True,
    coast_version="measures-v2",
    no_coast=True,
    region=region,
    cmap="../plotting/tmp2.cpt",
    cpt_lims=lims2,
    cbar_label="water column thickness (m)",
    # hist=True,
    cbar_yoffset=2.5,
    cbar_xoffset=-fig_width / 2,
    fig=fig,
    origin_shift="no_shift",
)
fig.grdcontour(
    grid=adusumilli_basal_melt_filt,
    interval="../plotting/tmp.cpt",
    annotation="n",
    pen="1p,+cl",
)
# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Satellite altimetry",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.text(
    position="TL",
    text="b",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
info = utils.set_proj(
    region,
    fig_height=10,
)
maps.add_scalebar(fig, region, info[1], scalebar_box="+gwhite", position="n.2/.07")
maps.add_north_arrow(
    fig,
    region,
    info[1],
    position="n.5/.07",
    # rose_box="+gwhite+c.01c/.01c",
    # rose_box="+p1p,white"
)

fig.show()

In [ ]:
region = list(inversion_region)
region[2] = region[2] - 100e3

fig_height = 10

fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=fig_height,
    transparency=50,
)
# make colormap
lims = [-2, 2]
# pygmt.grd2cpt(
#     grid=das_melt_grid,
#     cmap="balance+h0",
#     limit=lims,
#     # nlevels=11,
#     truncate=[-0.95, 0.95],
#     background="o",
#     output="../plotting/tmp.cpt",
# )
pygmt.makecpt(
    cmap="balance+h0",
    series=lims,
    truncate=[-0.95, 0.95],
    background="o",
    output="../plotting/tmp.cpt",
)
fig = maps.plot_grd(
    grid=das_melt_grid,
    fig_height=fig_height,
    region=region,
    cmap="../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="basal melt rate (m/yr)",
    hist=True,
    cbar_yoffset=1,
    fig=fig,
    origin_shift="no_shift",
    # transparency=100,
)

# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Airborne radar",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.text(
    position="TL",
    text="a",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)

fig_width = utils.get_fig_width(fig)


fig = maps.plot_grd(
    fetch.modis_moa(),
    image=True,
    cmap="gray",
    region=region,
    fig_height=fig_height,
    transparency=50,
    fig=fig,
    origin_shift="xshift",
)
# # make colormap
# lims = [-1, 1]
# pygmt.grd2cpt(
#     grid=adusumilli_basal_melt_filt,
#     cmap="balance+h0",
#     limit=lims,
#     # nlevels=11,
#     truncate=[-0.95, 0.95],
#     background="o",
#     output="../plotting/tmp.cpt",
# )
fig = maps.plot_grd(
    grid=adusumilli_basal_melt_filt,
    fig_height=fig_height,
    region=region,
    cmap="../plotting/tmp.cpt",
    cpt_lims=lims,
    cbar_label="basal melt rate (m/yr)",
    hist=True,
    cbar_yoffset=1,
    fig=fig,
    origin_shift="no_shift",
)

# plot titles
fig.text(
    position="TR",
    justify="TR",
    text="Satellite altimetry",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
fig.text(
    position="TL",
    text="b",
    fill="white",
    font="16p,Helvetica,black",
    offset="j.2/.2",
    clearance="+tO",
)
info = utils.set_proj(
    region,
    fig_height=10,
)
maps.add_scalebar(fig, region, info[1], scalebar_box="+gwhite", position="n.2/.07")
maps.add_north_arrow(
    fig,
    region,
    info[1],
    position="n.5/.07",
    # rose_box="+gwhite+c.01c/.01c",
    # rose_box="+p1p,white"
)

fig.show()

In [ ]:
import pooch

extensions = [
    # '.cpg',
    ".dbf",
    ".prj",
    ".shp",
    ".shx",
]

url = "https://github.com/mdtanker/RIS_basement_sediment/raw/v2.0/data/shapefiles/"


for i, j in itertools.product(["RIS_tight_outline"], extensions):
    p = pooch.retrieve(
        url=url + i + j,
        path=f"{pooch.os_cache('pooch')}/GRL_2022/shapefiles/",
        fname=i + j,
        known_hash=None,
        progressbar=True,
    )
    if p.endswith(".shp"):
        shp = p
shp = gpd.read_file(shp).geometry.buffer(40e3)
shp[0]

In [ ]:
grids = [
    merged_with_bedmachine.bed.rename("z"),
    adusumilli_basal_melt_filt.rename("z"),
]
fig = maps.plot_3d(
    grids=grids,
    cmaps=["batlowW", "balance+h0"],
    cpt_lims=[[-1200, -200], [-1, 1]],
    grd2cpt=False,
    exaggeration=[20, 1],
    # cbar_labels=["basement", "bed", "icebase", "surface"],
    # shp_mask="../plotting/RIS_outline.shp",
    shp_mask=shp,
    region=inversion_region,
    # cbar_xshift=[0, 0, ],
    cbar_yshift=[3, 6],
    zshifts=[20, 0],
    # transparencies=[0, 0, ],
    shading=True,
)

fig.show()

## Focusing on specific regions

### KIS

In [ ]:
# polygon = regions.draw_region()

In [ ]:
# region = utils.polygon_to_region(polygon)
region = (
    -412508.1190319692,
    -205692.1135234844,
    -846114.1423739997,
    -660770.0816229897,
)
region

In [ ]:
# KIS-1 is at 82.77S, 156.57E
# KIS-2 is at 82.47S, 152.48E
# df = pd.DataFrame({"lat":[-82.77, -82.47],"lon":[-156.57,-152.48],"site":["KIS1", "KIS2"]})
# df = utils.latlon_to_epsg3031(df,)

KIS = pd.read_csv("/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_KIS.csv")
KIS = utils.latlon_to_epsg3031(KIS, input=["Longitude", "Latitude"])
KIS
# pd.read_csv("/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_DD_2feb22.csv")

In [ ]:
df = KIS[KIS.Station != "KIS"]
KIS1 = df[df.Station.astype("int") < 2000]
KIS2 = df[(df.Station.astype("int") >= 2000) & (df.Station.astype("int") < 3000)]
KIS3 = df[df.Station.astype("int") >= 3000]

In [ ]:
fig = maps.plot_grd(
    merged_with_bedmachine.bed,
    region=region,
    fig_height=10,
    coast=True,
    coast_version="measures-v2",
    cmap="batlowW",
    cpt_lims=[-800, -400],
)

# fig.plot(x = KIS.x, y=KIS.y, style="c.05c", fill="black")
# fig.plot(x = KIS[KIS.Station=="KIS"].x, y=KIS[KIS.Station=="KIS"].y, style="c.05c", fill="red")

fig.plot(
    x=KIS1.x,
    y=KIS1.y,
    style="c.08c",
    fill="red",
    label="KIS1",
)
fig.plot(
    x=KIS2.x,
    y=KIS2.y,
    style="c.06c",
    fill="black",
    label="KIS2",
)
fig.plot(
    x=KIS3.x,
    y=KIS3.y,
    style="c.04c",
    fill="blue",
    label="KIS3",
)
fig.legend()
fig.show()

In [ ]:
profiles.sample_grids(
    KIS3,
    merged_with_bedmachine.bed,
    "sampled_bed",
    coord_names=["x", "y"],
).sampled_bed.describe()

In [ ]:
profiles.sample_grids(
    KIS3,
    merged_with_bedmachine.bed_error,
    "sampled_bed",
    coord_names=["x", "y"],
).sampled_bed.describe()

In [ ]:
profiles.sample_grids(
    KIS3,
    bedmachine_bed_full,
    "sampled_bed",
    coord_names=["x", "y"],
).sampled_bed.describe()

In [ ]:
profiles.sample_grids(
    KIS3,
    bedmachine_bed_error_full,
    "sampled_bed",
    coord_names=["x", "y"],
).sampled_bed.describe()

In [ ]:
grids = [
    merged_with_bedmachine.bed,
    MC_bed_ocean_draft,
    bedmachine_bed_full,
    bedmachine_ocean_draft,
]
titles = [
    "Inverted bathymetry",
    "Ocean draft",
    "Inverted bathymetry (BedMachine)",
    "Ocean draft (BedMachine)",
]
cbar_labels = ["elevation (m)", "thickness (m)", "elevation (m)", "thickness (m)"]
cmaps = [
    "batlowW",
    "rain",
    "batlowW",
    "rain",
]
cpt_lims = [
    [-800, -400],
    [0, 300],
    [-800, -400],
    [0, 300],
]
fig_height = 10

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
        yshift_amount = 1
    elif i == 2:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = -1
        yshift_amount = -1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1
        yshift_amount = 1

    g = g.sel(
        easting=slice(region[0], region[1]),
        northing=slice(region[2], region[3]),
    )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmaps[i],
        cpt_lims=cpt_lims[i],
        # robust=True,
        cbar_label=cbar_labels[i],
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.12c",
        fill="white",
        pen="1p,black",
        label="constraint point",
    )
    fig.plot(x=KIS.x, y=KIS.y, style="c.05c", fill="black", label="KIS survey")

    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 0:
        fig.legend(
            position="JBR+jBR",
            box="+gwhite",
        )
        info = utils.set_proj(
            region,
            fig_height=fig_height,
        )
        maps.add_scalebar(fig, region, info[1], position="n.15/.07")
        maps.add_north_arrow(fig, region, info[1], position="n.35/.07")


fig.show()

### Disco Deep

In [ ]:
# polygon = regions.draw_region()

In [ ]:
# region = utils.polygon_to_region(polygon)
region = (
    213240.66542271763,
    387270.82749028463,
    -1229482.699344385,
    -1022634.6107080313,
)
region

In [ ]:
DD = pd.read_csv(
    "/Volumes/arc_04/tankerma/RIS_grav_bath_data/raw_data/K863_DD_2feb22.csv"
)
DD = utils.latlon_to_epsg3031(DD, input=["Longitude", "Latitude"])
df = DD[DD.Name.str.contains("L")]
# DD.Name.unique()
DD = DD.sort_values(by="x")
DD

In [ ]:
DD.iloc[-1]

In [ ]:
g = utils.mask_from_shp(
    gpd.read_file("../plotting/RIS_outline.shp").buffer(-20e3),
    xr_grid=merged_with_bedmachine.bed,
    masked=True,
    invert=False,
)
np.nanmin(g)

In [ ]:
grd = pygmt.grdclip(
    merged_with_bedmachine.bed,
    below=[-1300, np.nan],
)
maps.plot_grd(
    grd,
    region=regions.ross_ice_shelf,
    fig_height=10,
    coast=True,
    coast_version="measures-v2",
    cmap="batlowW",
    cpt_lims=[-1400, -400],
)

In [ ]:
fig = maps.plot_grd(
    merged_with_bedmachine.bed,
    region=region,
    fig_height=10,
    coast=True,
    coast_version="measures-v2",
    cmap="batlowW",
    cpt_lims=[-1400, -400],
)
# fig.grdcontour(merged_with_bedmachine.bed, interval=50, limit=[-1410, -1290])
fig.grdcontour(merged_with_bedmachine.bed, annotation="1200")
fig.grdcontour(merged_with_bedmachine.bed, annotation="1300")

fig.plot(
    x=DD.x,
    y=DD.y,
    style="c.06c",
    fill="black",
    label="DiscoDeep",
)
fig.plot(
    x=DD.iloc[-1].x,
    y=DD.iloc[-1].y,
    style="c.06c",
    fill="red",
    # label="DiscoDeep",
)

df = DD[DD.Name.str.contains("L")]
fig.plot(
    x=df.x,
    y=df.y,
    style="c.04c",
    fill="blue",
    # label="DiscoDeep",
)
fig.legend()
fig.show()

In [ ]:
profiles.sample_grids(
    DD,
    merged_with_bedmachine.bed,
    "sampled_bed",
    coord_names=["x", "y"],
)  # .sampled_bed.describe()

In [ ]:
profiles.sample_grids(
    DD,
    bedmachine_bed_full,
    "sampled_bed",
    coord_names=["x", "y"],
)  # .sampled_bed.describe()

In [ ]:
grids = [
    merged_with_bedmachine.bed,
    MC_bed_ocean_draft,
    bedmachine_bed_full,
    bedmachine_ocean_draft,
]
titles = [
    "Inverted bathymetry",
    "Ocean draft",
    "Inverted bathymetry (BedMachine)",
    "Ocean draft (BedMachine)",
]
cbar_labels = ["elevation (m)", "thickness (m)", "elevation (m)", "thickness (m)"]
cmaps = [
    "batlowW",
    "rain",
    "batlowW",
    "rain",
]
cpt_lims = [
    [-1400, -400],
    [0, 800],
    [-1400, -400],
    [0, 800],
]
fig_height = 12

for i, g in enumerate(grids):
    if i == 0:
        fig = None
        origin_shift = "initialize"
        xshift_amount = 1
        yshift_amount = 1
    elif i == 2:
        fig = fig
        origin_shift = "both_shift"
        xshift_amount = -1
        yshift_amount = -1
    else:
        fig = fig
        origin_shift = "xshift"
        xshift_amount = 1
        yshift_amount = 1

    g = g.sel(
        easting=slice(region[0], region[1]),
        northing=slice(region[2], region[3]),
    )

    fig = maps.plot_grd(
        fetch.modis_moa(),
        image=True,
        cmap="gray",
        region=region,
        fig_height=fig_height,
        transparency=50,
        fig=fig,
        origin_shift=origin_shift,
        xshift_amount=xshift_amount,
        yshift_amount=yshift_amount,
    )

    fig = maps.plot_grd(
        grid=g,
        fig_height=fig_height,
        coast=True,
        coast_version="measures-v2",
        region=region,
        cmap=cmaps[i],
        cpt_lims=cpt_lims[i],
        # robust=True,
        cbar_label=cbar_labels[i],
        hist=True,
        cbar_yoffset=1,
        fig=fig,
        origin_shift="no_shift",
    )
    fig.grdcontour(g)

    # plot titles
    fig.text(
        position="TR",
        justify="TR",
        text=titles[i],
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    fig.plot(
        x=constraints[constraints.inside].easting,
        y=constraints[constraints.inside].northing,
        style="c.12c",
        fill="white",
        pen="1p,black",
        label="constraint point",
    )
    # fig.plot(x = KIS.x, y=KIS.y, style="c.05c", fill="black", label="KIS survey")

    fig.text(
        position="TL",
        text=f"{string.ascii_lowercase[i]}",
        fill="white",
        font="16p,Helvetica,black",
        offset="j.2/.2",
        clearance="+tO",
    )
    if i == 0:
        fig.legend(
            position="JBR+jBR",
            box="+gwhite",
        )
        info = utils.set_proj(
            region,
            fig_height=fig_height,
        )
        maps.add_scalebar(fig, region, info[1], position="n.15/.07")
        maps.add_north_arrow(fig, region, info[1], position="n.35/.07")


fig.show()

# Profiles

In [ ]:
# lines = profiles.draw_lines(points=constraints[constraints.inside].rename(columns={"easting":"x", "northing":"y"}))

In [ ]:
# get clicked points
df = utils.shapes_to_df(lines)

# extract first and last
a = (df.x.iloc[0], df.y.iloc[0])
b = (df.x.iloc[-1], df.y.iloc[-1])

# # siple coast
# a, b = ((-514139.22589550324, -886012.3396426054),
# (-30871.591933855758, -595196.2984418197))

# # ice front to CIR
# a,b = ((48403.646330947464, -1387918.0291082778),
#  (-221877.26567443524, -613795.4844574957))

# # Roosevelt Island to TAM
# a,b = ((-489471.998996549, -1177190.3876565588),
#  (190464.00058142765, -777830.3659641005))

# # KIS to Ross Island
# a,b = ((-391311.08538905036, -713591.4759308188),
# (288280.6584588745, -1311607.5333427137))

# South of Roosevelt to TAM
# a,b = ((-571375.4234697601, -1051511.5422168665),
#  (132779.8923736067, -719735.4502564461))

a, b

In [ ]:
# # calving front near Roosevelt to CIR
# df.to_csv("plotting/up_nimrod_profile.csv", index=False,)
# df = pd.read_csv("plotting/up_nimrod_profile.csv")

# # pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,

#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     map_cmap="batlowW",
#     # map_cmap="plotting/tmp.cpt",
#     map_buffer=0.2,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],
#     # fillnans=False,
# )
# fig.show()

In [ ]:
# # calving front near Roosevelt to CIR
# # df.to_csv("plotting/RI_calving_front_to_CIR_profile.csv", index=False,)
# df = pd.read_csv("plotting/RI_calving_front_to_CIR_profile.csv")

# # pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,

#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     map_cmap="batlowW",
#     # map_cmap="plotting/tmp.cpt",
#     map_buffer=0.2,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],
#     # fillnans=False,
# )
# fig.show()

In [ ]:
# # calving front to byrd glacier
# df.to_csv("plotting/up_byrd_profile.csv", index=False,)
# df = pd.read_csv("plotting/up_byrd_profile.csv")

# pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,

#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     # map_cmap="batlowW",
#     map_cmap="plotting/tmp.cpt",
#     map_buffer=0.2,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],

# )
# fig.show()

In [ ]:
# # Disco deep traverse points
# # df.to_csv("plotting/disco_deep_traverse_profile.csv", index=False,)
# df = pd.read_csv("plotting/disco_deep_traverse_profile.csv")

# pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,
#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     # map_cmap="batlowW",
#     map_cmap="plotting/tmp.cpt",
#     map_buffer=0.2,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],

# )
# fig.show()

In [ ]:
# # across CIR
# # a,b = ((-395263.9962105298, -737706.6673521262),
# #  (169983.9921897897, -749994.681560052))
# # df.to_csv("plotting/CIR_profile.csv", index=False,)
# df = pd.read_csv("plotting/CIR_profile.csv")

# pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,
#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     # map_cmap="batlowW",
#     map_cmap="plotting/tmp.cpt",
#     map_buffer=0.2,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],

# )
# fig.show()

In [ ]:
# # calving front
# # df.to_csv("plotting/calving_front_profile.csv", index=False,)
# df = pd.read_csv("plotting/calving_front_profile.csv")

# pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,
#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     # map_cmap="batlowW",
#     map_cmap="plotting/tmp.cpt",
#     map_buffer=0.01,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],

# )
# fig.show()

In [ ]:
# # calving front
# # df.to_csv("plotting/groundingline_profile.csv", index=False,)
# df = pd.read_csv("plotting/groundingline_profile.csv")

# pygmt.grd2cpt(cmap="batlowW", grid=final_bed, output="plotting/tmp.cpt")

# fig = profiles.plot_profile(

#     # "points",
#     # start=a,
#     # stop=b,
#     "polyline",
#     polyline=df,
#     num=1000,

#     # add_map=True,
#     map_background=final_bed,
#     # map_cmap="batlowW",
#     map_cmap="plotting/tmp.cpt",
#     map_buffer=0.01,
#     map_points=constraints[constraints.inside].rename(
#         columns={"easting": "x", "northing": "y"}
#     ),

#     inset_width=.15,
#     inset_pos="BL",
#     data_height=5,
#     fig_height=6,

#     # subplot_orientation="horizontal",
#     subplot_orientation='vertical',

#     # data_dict=data_dict,
#     layers_dict=layers_dict,
#     layers_legend=True,
#     layers_legend_loc="jTR",
#     layers_legend_box="+gwhite",
#     share_yaxis=True,
#     layer_transparency=[40,40,0,60,40],

# )
# fig.show()